# 한글폰트 설치 - 설치 후 세션 재시작 필요

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
!pip install shapely
!pip install joblib
!pip install geopandas
!pip install folium mapclassify


# 라이브러리 로드

In [ ]:
import pandas as pd
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.font_manager
font_manager = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')
import seaborn as sns
import folium
import geopandas as gpd
from shapely.wkt import loads
# from shapely import wkt
from fiona.crs import from_string
import math
from folium.plugins import AntPath
import warnings
warnings.filterwarnings('ignore')

# from adjustText import adjust_text

# 시각화 함수

## 지도 시각화


In [ ]:
def map(df, key, value, legend_name='test', tootip=None):
  # Folium 지도 생성
  style = {'fillColor': '#00000000', 'color': '#00000000'}
  map_heatmap = folium.Map(location=[37.4, 127.5], zoom_start=10)

  cp = folium.Choropleth(
    geo_data=df, # 경계선 좌표값이 담긴 데이터
    data=df, # Series or DataFrame 넣으면 된다
    columns=[key, value], # DataFrame의 어떤 columns을 넣을지
    key_on=f'feature.properties.{key}',
    line_color = 'black',
    fill_color='YlOrRd',
    fill_opacity=0.5, # 색 투명도
    line_opacity=0.5, # 선 투명도
    legend_name=legend_name # 범례
    ).add_to(map_heatmap)

  if tootip != None:
    folium.GeoJson(
    df,
    tooltip=folium.GeoJsonTooltip(fields=tootip),
    style_function=lambda x: style
    ).add_to(map_heatmap)

  # 지도 표시
  return display(map_heatmap)

In [ ]:
def map_3(visual_o, visual_d):

  def calculate_rotation(start_point, end_point):
    dx = end_point[0] - start_point[0]
    dy = end_point[1] - start_point[1]
    angle = math.atan2(dy, dx)  # 라디안 단위의 각도
    angle_degrees = math.degrees(angle)  # 도 단위로 변환
    return angle_degrees

  # 맵 생성
  m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)

  # # 출발지와 도착지 좌표
  start_point = list(visual_o.loc[top_1, 'geometry'].centroid.coords)[0][::-1]
  end_point = list(visual_d.loc[top_1, 'geometry'].centroid.coords)[0][::-1]

  # 회전 각도 계산
  rotation = calculate_rotation(start_point, end_point)

  cp = folium.Choropleth(
    geo_data=visual_o, # 경계선 좌표값이 담긴 데이터
    data=visual_o, # Series or DataFrame 넣으면 된다
    columns=['O_CTY_CD', 'CNT'], # DataFrame의 어떤 columns을 넣을지
    key_on=f'feature.properties.O_CTY_CD',
    line_color = 'black',
    fill_color='YlOrRd',
    fill_opacity=0.5, # 색 투명도
    line_opacity=0.5, # 선 투명도
    legend_name='유출입' # 범례
    ).add_to(m)

  folium.GeoJson(
  visual_o,
  tooltip=folium.GeoJsonTooltip(fields= ['O_CTY_NM', 'CNT'])
  ).add_to(m)

  for child in cp._children:
    if child.startswith("color_map"):
        del cp._children[child]

  cp = folium.Choropleth(
    geo_data=visual_d, # 경계선 좌표값이 담긴 데이터
    data=visual_d, # Series or DataFrame 넣으면 된다
    columns=['D_CTY_CD', 'CNT'], # DataFrame의 어떤 columns을 넣을지
    key_on=f'feature.properties.D_CTY_CD',
    line_color = 'black',
    fill_color='YlOrRd',
    fill_opacity=0.5, # 색 투명도
    line_opacity=0.5, # 선 투명도
    # legend_name='유출입' # 범례
    ).add_to(m)

  folium.GeoJson(
  visual_d,
  tooltip=folium.GeoJsonTooltip(fields= ['D_CTY_NM', 'CNT'])
  ).add_to(m)

  for child in cp._children:
    if child.startswith("color_map"):
      del cp._children[child]

  # RegularPolygonMarker 추가 (색상 검은색으로 설정)
  folium.RegularPolygonMarker(location=end_point, number_of_sides=3, radius=15, rotation=rotation-90, color='black', fill_color='black').add_to(m)

  # AntPath로 라인의 방향성 나타내기 (선 색상 검은색으로 설정)
  ant_path = AntPath(locations=[start_point, end_point], dash_array=[10, 20], delay=1000, weight=6, color='black')
  ant_path.add_to(m)

  # m.get_root().html.add_child('<script>document.getElementsByClassName("leaflet-bottom leaflet-left")[0].style.display = "none";</script>')


  # 맵 표시
  display(m)


## 바그래프 시각화

In [ ]:
def create_barplot(data, x, y, title=None, xlabel=None, ylabel=None, legend=None):
  plt.figure(figsize=(12,6))

  if title:
      plt.title(title, fontsize = 17)
  if xlabel:
      plt.xlabel(xlabel, fontsize=15)
  if ylabel:
      plt.ylabel(ylabel, fontsize=15)
  if legend:
    plt.legend(loc='upper right')

  ax = sns.barplot(data=data, x=x, y=y, hue=legend, errorbar=None)

  plt.ticklabel_format(axis='y',useOffset=False, style='plain')

  max_y = data[y].max()  # y 열의 최댓값 계산
  plt.ylim(0, max_y * 1.3)  # 최댓값의 30%를 여유 공간으로 추가

  plt.show()

## 라인그래프 시각화

In [ ]:
def create_lineplot(data, x, y, title=None, xlabel=None, ylabel=None, legend=None):
  plt.figure(figsize=(14,7))

  if title:
      plt.title(title, fontsize = 17)
  if xlabel:
      plt.xlabel(xlabel, fontsize=15)
  if ylabel:
      plt.ylabel(ylabel, fontsize=15)
  if legend:
    plt.legend(loc='upper right')

  sns.lineplot(data=data, x=x, y=y, hue=legend)

  plt.xticks(data[x])
  plt.ticklabel_format(axis='y',useOffset=False, style='plain')

  max_y = data[y].max()  # y 열의 최댓값 계산
  plt.ylim(0, max_y * 1.3)  # 최댓값의 30%를 여유 공간으로 추가

  plt.show()

# 시군구별 유동인구 시각화

In [ ]:
cty_shp = pd.read_csv('/content/drive/MyDrive/시군구 경계/TBSHP_LH_CTY_202404081545.csv', encoding='UTF-8')
cty_shp['SGG_CD'] = cty_shp['SGG_CD'].astype('str')
cty_shp['GEOMETRY'] = cty_shp['GEOMETRY'].astype('str')
cty_shp['geometry'] = cty_shp['GEOMETRY'].apply(lambda wkt: loads(wkt))
cty_shp = gpd.GeoDataFrame(cty_shp, geometry = 'geometry').set_crs("EPSG:5181")
cty_shp = cty_shp.drop(columns = ['GEOMETRY'])
cty_shp = cty_shp[['SGG_CD', 'geometry']]
cty_shp = cty_shp.to_crs("EPSG:4326")

In [ ]:
wdata = pd.read_csv('/content/drive/MyDrive/GG_SGG_FLOWPOP.csv')
data['year_mm'] = data['ETL_YMD'].astype(str).str.slice(start=0, stop=6)
data['M_10_CNT_age'] = data['M_10_CNT'] +  data['M_15_CNT']
data['M_20_CNT_age'] = data['M_20_CNT'] +  data['M_25_CNT']
data['M_30_CNT_age'] = data['M_30_CNT'] +  data['M_35_CNT']
data['M_40_CNT_age'] = data['M_40_CNT'] +  data['M_45_CNT']
data['M_50_CNT_age'] = data['M_50_CNT'] +  data['M_55_CNT']
data['M_60_CNT_age'] = data['M_60_CNT'] +  data['M_65_CNT']
data['M_70_CNT_age'] = data['M_70_CNT']
data['W_10_CNT_age'] = data['W_10_CNT'] +  data['W_15_CNT']
data['W_20_CNT_age'] = data['W_20_CNT'] +  data['W_25_CNT']
data['W_30_CNT_age'] = data['W_30_CNT'] +  data['W_35_CNT']
data['W_40_CNT_age'] = data['W_40_CNT'] +  data['W_45_CNT']
data['W_50_CNT_age'] = data['W_50_CNT'] +  data['W_55_CNT']
data['W_60_CNT_age'] = data['W_60_CNT'] +  data['W_65_CNT']
data['W_70_CNT_age'] = data['W_70_CNT']
data['M_CNT_age'] = data['M_10_CNT_age'] + data['M_20_CNT_age'] + data['M_30_CNT_age'] + data['M_40_CNT_age'] + data['M_50_CNT_age'] + data['M_60_CNT_age'] + data['M_70_CNT_age']
data['W_CNT_age'] = data['W_10_CNT_age'] + data['W_20_CNT_age'] + data['W_30_CNT_age'] + data['W_40_CNT_age'] + data['W_50_CNT_age'] + data['W_60_CNT_age'] + data['W_70_CNT_age']
data['total'] = data['M_CNT_age'] + data['W_CNT_age']
data.loc[data['CTY_CD'].isin([41194, 41192, 41196]), 'CTY_CD'] = 41190
data.loc[data['CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'CTY_NM'] = '부천시'

In [ ]:
group_data = data[['year_mm', 'CTY_CD', 'CTY_NM', 'total']]
group_data = group_data.groupby(by=['year_mm', 'CTY_CD', 'CTY_NM'])['total'].sum().reset_index()
group_data['CTY_CD'] = group_data['CTY_CD'].astype(str)
concat_data = pd.merge(group_data, cty_shp, left_on = 'CTY_CD', right_on = 'SGG_CD', how = 'left')
concat_data = gpd.GeoDataFrame(concat_data, geometry='geometry')
concat_data = concat_data[~concat_data['SGG_CD'].isnull()]

In [ ]:
concat_01 = concat_data[concat_data['year_mm'] == '202401']
map(concat_01, 'CTY_CD', 'total', '유동인구', tootip = ['CTY_NM', 'total'])

In [ ]:
concat_02 = concat_data[concat_data['year_mm'] == '202402']
map(concat_02, 'CTY_CD', 'total', '유동인구', tootip = ['CTY_NM', 'total'])

# 시군구 유동인구 데이터 정제

In [ ]:
# data = df.copy()
data['year'] = data['ETL_YMD'].astype(str).str.slice(start=0, stop=4)
data['mm'] = data['ETL_YMD'].astype(str).str.slice(start=4, stop=6)
convert_list = ['M_10_CNT' ,'M_15_CNT' ,'M_20_CNT' ,'M_25_CNT' ,'M_30_CNT' ,'M_35_CNT' ,'M_40_CNT' ,'M_45_CNT' ,'M_50_CNT' ,'M_55_CNT' ,'M_60_CNT' ,'M_65_CNT' ,'M_70_CNT' ,'W_10_CNT' ,'W_15_CNT' ,'W_20_CNT' ,'W_25_CNT' ,'W_30_CNT' ,'W_35_CNT' ,'W_40_CNT' ,'W_45_CNT' ,'W_50_CNT' ,'W_55_CNT' ,'W_60_CNT' ,'W_65_CNT' ,'W_70_CNT']
for colum_name in convert_list:
  data[colum_name] = data[colum_name].round().astype(int)

In [ ]:
data['M_10_CNT_age'] = data['M_10_CNT'] +  data['M_15_CNT']
data['M_20_CNT_age'] = data['M_20_CNT'] +  data['M_25_CNT']
data['M_30_CNT_age'] = data['M_30_CNT'] +  data['M_35_CNT']
data['M_40_CNT_age'] = data['M_40_CNT'] +  data['M_45_CNT']
data['M_50_CNT_age'] = data['M_50_CNT'] +  data['M_55_CNT']
data['M_60_CNT_age'] = data['M_60_CNT'] +  data['M_65_CNT']
data['M_70_CNT_age'] = data['M_70_CNT']
data['W_10_CNT_age'] = data['W_10_CNT'] +  data['W_15_CNT']
data['W_20_CNT_age'] = data['W_20_CNT'] +  data['W_25_CNT']
data['W_30_CNT_age'] = data['W_30_CNT'] +  data['W_35_CNT']
data['W_40_CNT_age'] = data['W_40_CNT'] +  data['W_45_CNT']
data['W_50_CNT_age'] = data['W_50_CNT'] +  data['W_55_CNT']
data['W_60_CNT_age'] = data['W_60_CNT'] +  data['W_65_CNT']
data['W_70_CNT_age'] = data['W_70_CNT']
data['M_CNT_age'] = data['M_10_CNT_age'] + data['M_20_CNT_age'] + data['M_30_CNT_age'] + data['M_40_CNT_age'] + data['M_50_CNT_age'] + data['M_60_CNT_age'] + data['M_70_CNT_age']
data['W_CNT_age'] = data['W_10_CNT_age'] + data['W_20_CNT_age'] + data['W_30_CNT_age'] + data['W_40_CNT_age'] + data['W_50_CNT_age'] + data['W_60_CNT_age'] + data['W_70_CNT_age']
data['year_mm'] = data['year'] + data['mm']

# 유동인구가 가장 많은 시군구(Top 5) 연령별·성별 유동인구


In [ ]:
melt_data = data[['year_mm', 'CTY_NM', 'M_10_CNT_age', 'M_20_CNT_age', 'M_30_CNT_age', 'M_40_CNT_age', 'M_50_CNT_age', 'M_60_CNT_age', 'M_70_CNT_age', 'W_10_CNT_age', 'W_20_CNT_age', 'W_30_CNT_age', 'W_40_CNT_age', 'W_50_CNT_age', 'W_60_CNT_age', 'W_70_CNT_age']]
melt_df = pd.melt(melt_data, id_vars=['year_mm', 'CTY_NM'], var_name=['temp'], value_name='Value')
year_mm_list = melt_df['year_mm'].sort_values().drop_duplicates().to_list()

year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = melt_df[melt_df['year_mm'] == year_mm]
  temp = temp.groupby(by=['year_mm', 'CTY_NM'])['Value'].sum().reset_index()
  temp = temp.sort_values(by='Value', ascending=False)
  temp = temp[:5]
  temp_list = temp['CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list
# year_mm_si_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = data[data['year_mm'] == key]
  temp_df = temp_df[temp_df['CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])

In [ ]:
melt_data = concat_data[['year_mm', 'CTY_NM', 'M_10_CNT_age', 'M_20_CNT_age', 'M_30_CNT_age', 'M_40_CNT_age', 'M_50_CNT_age', 'M_60_CNT_age', 'M_70_CNT_age', 'W_10_CNT_age', 'W_20_CNT_age', 'W_30_CNT_age', 'W_40_CNT_age', 'W_50_CNT_age', 'W_60_CNT_age', 'W_70_CNT_age']]
melt_df = pd.melt(melt_data, id_vars=['year_mm', 'CTY_NM'], var_name=['temp'], value_name='Value')
melt_df['gender'] = melt_df['temp'].str.split('_').str[0]
melt_df['age'] = melt_df['temp'].str.split('_').str[1].astype(int)

In [ ]:
group_data = melt_df.groupby(['year_mm', 'CTY_NM', 'age', 'gender'])['Value'].sum().reset_index()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['year_mm'] == key_2]
  temp = temp.sort_values(by=['CTY_NM', 'age'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['year_mm'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['CTY_NM'] == si]
    create_barplot(visual_si_data, x='age', y='Value', title=f'{key} {si}·연령대별 유동인구', xlabel='연령대', ylabel='유동인구', legend='gender')

# 유동인구가 가장 많은 시군구(TOP 5) 시간대별·성별 유동인구

In [ ]:
melt_data = concat_data[['year_mm', 'TIME_CD', 'CTY_NM', 'M_10_CNT_age', 'M_20_CNT_age', 'M_30_CNT_age', 'M_40_CNT_age', 'M_50_CNT_age', 'M_60_CNT_age', 'M_70_CNT_age', 'W_10_CNT_age', 'W_20_CNT_age', 'W_30_CNT_age', 'W_40_CNT_age', 'W_50_CNT_age', 'W_60_CNT_age', 'W_70_CNT_age']]
melt_df = pd.melt(melt_data, id_vars=['year_mm','TIME_CD', 'CTY_NM'], var_name=['temp'], value_name='Value')
melt_df['gender'] = melt_df['temp'].str.split('_').str[0]

group_data = melt_df.groupby(['year_mm', 'TIME_CD', 'CTY_NM', 'gender'])['Value'].sum().reset_index()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['year_mm'] == key_2]
  temp = temp.sort_values(by=['CTY_NM', 'TIME_CD'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['year_mm'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['CTY_NM'] == si]
    create_lineplot(visual_si_data, x='TIME_CD', y='Value', title=f'{key} {si}·시간대별 유동인구', xlabel='시간대', ylabel='유동인구', legend='gender')

# 1월 시군구별 내국인 및 외국인 유동인구수

In [ ]:
sum_df = data.groupby(['CTY_CD', 'year_mm', 'FORN_GB'])['total'].sum().reset_index()
sum_df['CTY_CD'] = sum_df['CTY_CD'].astype(str)
concat_data = pd.merge(sum_df, cty_shp, left_on = 'CTY_CD', right_on = 'SGG_CD', how = 'left')
concat_data_01 = concat_data[concat_data['year_mm'] == '202401']

concat_data_01_in = concat_data_01[concat_data_01['FORN_GB'] == '내국인']
concat_data_01_in = concat_data_01_in.reset_index(drop=1)
concat_data_01_in = gpd.GeoDataFrame(concat_data_01_in, geometry=concat_data_01_in['geometry'], crs="EPSG:4326")
map(concat_data_01_in, 'CTY_CD', 'total', '유동인구', tootip = ['total'])

In [ ]:
concat_data_01_out = concat_data_01[concat_data_01['FORN_GB'] == '외국인']
concat_data_01_out = concat_data_01_out.reset_index(drop=1)
concat_data_01_out = gpd.GeoDataFrame(concat_data_01_out, geometry=concat_data_01_out['geometry'], crs="EPSG:4326")
map(concat_data_01_out, 'CTY_CD', 'total', '유동인구', tootip = ['total'])

# 2월 시군구별 내국인 및 외국인 유동인구수

In [ ]:
concat_data_02 = concat_data[concat_data['year_mm'] == '202402']

concat_data_02_in = concat_data_02[concat_data_02['FORN_GB'] == '내국인']
concat_data_02_in = concat_data_02_in.reset_index(drop=1)
concat_data_02_in = gpd.GeoDataFrame(concat_data_02_in, geometry=concat_data_02_in['geometry'], crs="EPSG:4326")
map(concat_data_02_in, 'CTY_CD', 'total', '유동인구', tootip = ['total'])

In [ ]:
concat_data_02_out = concat_data_02[concat_data_02['FORN_GB'] == '외국인']
concat_data_02_out = concat_data_02_out.reset_index(drop=1)
concat_data_02_out = gpd.GeoDataFrame(concat_data_02_out, geometry=concat_data_02_out['geometry'], crs="EPSG:4326")
map(concat_data_02_out, 'CTY_CD', 'total', '유동인구', tootip = ['total'])

# 행정동별 유동인구 시각화

In [ ]:
# 행정동별 유동인구 데이터
df_admi = pd.read_csv('/content/drive/MyDrive/GG_ADMI_FLOWPOP.csv')
df_admi['ADMI_CD'] = df_admi['ADMI_CD'].astype(str)
df_admi['ETL_YMD'] = df_admi['ETL_YMD'].astype(str)
df_admi['ymd'] = df_admi['ETL_YMD'].str[:6]
df_admi['name'] = df_admi['CTY_NM'] + ' ' + df_admi['ADMI_NM']

In [ ]:
df_admi['M_10_CNT_age'] = df_admi['M_10_CNT'] +  df_admi['M_15_CNT']
df_admi['M_20_CNT_age'] = df_admi['M_20_CNT'] +  df_admi['M_25_CNT']
df_admi['M_30_CNT_age'] = df_admi['M_30_CNT'] +  df_admi['M_35_CNT']
df_admi['M_40_CNT_age'] = df_admi['M_40_CNT'] +  df_admi['M_45_CNT']
df_admi['M_50_CNT_age'] = df_admi['M_50_CNT'] +  df_admi['M_55_CNT']
df_admi['M_60_CNT_age'] = df_admi['M_60_CNT'] +  df_admi['M_65_CNT']
df_admi['M_70_CNT_age'] = df_admi['M_70_CNT']
df_admi['W_10_CNT_age'] = df_admi['W_10_CNT'] +  df_admi['W_15_CNT']
df_admi['W_20_CNT_age'] = df_admi['W_20_CNT'] +  df_admi['W_25_CNT']
df_admi['W_30_CNT_age'] = df_admi['W_30_CNT'] +  df_admi['W_35_CNT']
df_admi['W_40_CNT_age'] = df_admi['W_40_CNT'] +  df_admi['W_45_CNT']
df_admi['W_50_CNT_age'] = df_admi['W_50_CNT'] +  df_admi['W_55_CNT']
df_admi['W_60_CNT_age'] = df_admi['W_60_CNT'] +  df_admi['W_65_CNT']
df_admi['W_70_CNT_age'] = df_admi['W_70_CNT']
df_admi['M_CNT_age'] = df_admi['M_10_CNT_age'] + df_admi['M_20_CNT_age'] + df_admi['M_30_CNT_age'] + df_admi['M_40_CNT_age'] + df_admi['M_50_CNT_age'] + df_admi['M_60_CNT_age'] + df_admi['M_70_CNT_age']
df_admi['W_CNT_age'] = df_admi['W_10_CNT_age'] + df_admi['W_20_CNT_age'] + df_admi['W_30_CNT_age'] + df_admi['W_40_CNT_age'] + df_admi['W_50_CNT_age'] + df_admi['W_60_CNT_age'] + df_admi['W_70_CNT_age']
df_admi['total'] = df_admi['M_CNT_age'] + df_admi['W_CNT_age']
df_admi['name'] = df_admi['CTY_NM'] + ' ' + df_admi['ADMI_NM']

In [ ]:
# 행정동별 유동인구 데이터
sum_df = df_admi.groupby(['name', 'ADMI_CD', 'ymd'])['total'].sum().reset_index()

cty_tmp_01 = sum_df[sum_df['ymd'] == '202401']
cty_tmp_01 = cty_tmp_01[['name', 'ADMI_CD', 'total', 'ymd']]

cty_tmp_02 = sum_df[sum_df['ymd'] == '202402']
cty_tmp_02 = cty_tmp_02[['name', 'ADMI_CD', 'total', 'ymd']]


In [ ]:
# 202401 행정동별 총 유동인구수
cty_tmp_01 = cty_tmp_01.sort_values(by='total', ascending=False)
cty_tmp_01.head()

In [ ]:
# 202402 행정동별 총 유동인구수
cty_tmp_02 = cty_tmp_02.sort_values(by='total', ascending=False)
cty_tmp_02.head()

# 행정동 유동인구 데이터 정제

In [ ]:
dong_data = df_admi.copy()
dong_data['year'] = dong_data['ETL_YMD'].astype(str).str.slice(start=0, stop=4)
dong_data['mm'] = dong_data['ETL_YMD'].astype(str).str.slice(start=4, stop=6)
dong_data['year_mm'] = dong_data['year'] + dong_data['mm']
dong_data['admi'] = dong_data['CTY_NM'] + ' ' + dong_data['ADMI_NM']

# 유동인구가 가장 많은 행정동(Top 5) 연령별·성별 유동인구


In [ ]:
melt_data = dong_data[['year_mm', 'admi', 'total']]

group_data = melt_data.groupby(['year_mm', 'admi'])['total'].sum().reset_index()
group_data.head()
year_mm_list = melt_data['year_mm'].drop_duplicates().sort_values().to_list()

year_mm_admi_dic = {}
for year_mm in year_mm_list:
  temp = melt_data[melt_data['year_mm'] == year_mm]
  temp = temp.groupby(by=['year_mm', 'admi'])['total'].sum().reset_index()
  temp = temp.sort_values(by='total', ascending=False)
  temp = temp[:5]
  temp_list = temp['admi'].to_list()
  year_mm_admi_dic[year_mm] = temp_list
# year_mm_admi_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_admi_dic.items():
  temp_df = dong_data[dong_data['year_mm'] == key]
  temp_df = temp_df[temp_df['admi'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])

In [ ]:
melt_data = concat_data[['year_mm', 'admi', 'M_10_CNT_age', 'M_20_CNT_age', 'M_30_CNT_age', 'M_40_CNT_age', 'M_50_CNT_age', 'M_60_CNT_age', 'M_70_CNT_age', 'W_10_CNT_age', 'W_20_CNT_age', 'W_30_CNT_age', 'W_40_CNT_age', 'W_50_CNT_age', 'W_60_CNT_age', 'W_70_CNT_age']]
melt_df = pd.melt(melt_data, id_vars=['year_mm', 'admi'], var_name=['temp'], value_name='Value')
melt_df['gender'] = melt_df['temp'].str.split('_').str[0]
melt_df['age'] = melt_df['temp'].str.split('_').str[1].astype(int)

In [ ]:
group_data = melt_df.groupby(['year_mm', 'admi', 'age', 'gender'])['Value'].sum().reset_index()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_admi_dic.items():
  temp = group_data[group_data['year_mm'] == key_2]
  temp = temp.sort_values(by=['admi', 'age'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_admi_dic.items():
  visual_data = sort_df[sort_df['year_mm'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['admi'] == si]
    create_barplot(visual_si_data, x='age', y='Value', title=f'{key} {si}·연령대별 유동인구', xlabel='연령대', ylabel='유동인구', legend='gender')

# 유동인구가 가장 많은 행정동(TOP 5) 시간대별·성별 유동인구

In [ ]:
melt_data = concat_data[['year_mm', 'TIME_CD', 'admi', 'M_10_CNT_age', 'M_20_CNT_age', 'M_30_CNT_age', 'M_40_CNT_age', 'M_50_CNT_age', 'M_60_CNT_age', 'M_70_CNT_age', 'W_10_CNT_age', 'W_20_CNT_age', 'W_30_CNT_age', 'W_40_CNT_age', 'W_50_CNT_age', 'W_60_CNT_age', 'W_70_CNT_age']]
melt_df = pd.melt(melt_data, id_vars=['year_mm', 'TIME_CD', 'admi'], var_name=['temp'], value_name='Value')
melt_df['gender'] = melt_df['temp'].str.split('_').str[0]

group_data = melt_df.groupby(['year_mm', 'TIME_CD', 'admi', 'gender'])['Value'].sum().reset_index()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_admi_dic.items():
  temp = group_data[group_data['year_mm'] == key_2]
  temp = temp.sort_values(by=['admi', 'TIME_CD'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df


In [ ]:
for key, si_list in year_mm_admi_dic.items():
  visual_data = sort_df[sort_df['year_mm'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['admi'] == si]
    create_lineplot(visual_si_data, x='TIME_CD', y='Value', title=f'{key} {si}·시간대별 유동인구', xlabel='시간대', ylabel='유동인구', legend='gender')

# 시군구별 유입 방문목적별 체류시간


In [ ]:
sgg_time_income_data = pd.read_csv('./drive/MyDrive/GG_TRANS_SEXAGE_DURATION_SGG_INFLOW.csv')
sgg_time_income_data['DURATION'] = sgg_time_income_data['DURATION'].astype(float)
sgg_time_income_data

In [ ]:
year_mm_list = sgg_time_income_data['ETL_YM'].sort_values().drop_duplicates().to_list()
year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = sgg_time_income_data[sgg_time_income_data['ETL_YM'] == year_mm]
  temp = temp.groupby(by=['ETL_YM', 'D_CTY_NM'])['CNT'].sum().reset_index()
  temp = temp.sort_values(by='CNT', ascending=False)
  temp = temp[:5]
  temp_list = temp['D_CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = sgg_time_income_data[sgg_time_income_data['ETL_YM'] == key]
  temp_df = temp_df[temp_df['D_CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])

concat_data['time'] = ''
concat_data.loc[concat_data['DURATION'] < 60, 'time'] = '1시간 미만'
concat_data.loc[(concat_data['DURATION'] >= 60) & (concat_data['DURATION'] < 120), 'time'] = '1시간 이상'
concat_data.loc[(concat_data['DURATION'] >= 120) & (concat_data['DURATION'] < 180), 'time'] = '2시간 이상'
concat_data.loc[concat_data['DURATION'] >= 180, 'time'] = '3시간 이상'

In [ ]:
concat_data

# 행정동별 국적별 유동인구 확인

In [ ]:
# 국적별 총 유동인구수 데이터

cty_forng_01 = df_admi[(df_admi['ymd'] == '202401') & (df_admi['FORN_GB'] == '내국인')]
cty_fornb_02 = df_admi[(df_admi['ymd'] == '202401') & (df_admi['FORN_GB'] == '외국인')]

cty_forn_01 = df_admi[(df_admi['ymd'] == '202402') & (df_admi['FORN_GB'] == '내국인')]
cty_forn_02 = df_admi[(df_admi['ymd'] == '202402') & (df_admi['FORN_GB'] == '외국인')]

In [ ]:
cty_forng_01 = cty_forng_01.groupby(by=['name', 'FORN_GB', 'ymd'])['total'].sum().reset_index()
cty_fornb_02 = cty_fornb_02.groupby(by=['name', 'FORN_GB', 'ymd'])['total'].sum().reset_index()
cty_forn_01 = cty_forn_01.groupby(by=['name', 'FORN_GB', 'ymd'])['total'].sum().reset_index()
cty_forn_02 = cty_forn_02.groupby(by=['name', 'FORN_GB', 'ymd'])['total'].sum().reset_index()

In [ ]:
# 202401 행정동별 내국인 유동인구수
cty_forng_01.head().sort_values(by='total', ascending=False)

In [ ]:
# 202402 행정동별 내국인 유동인구수
cty_forn_01.head().sort_values(by='total', ascending=False)

In [ ]:
# 202401 행정동별 외국인 유동인구수
cty_fornb_02.head().sort_values(by='total', ascending=False)

In [ ]:
# 202402 행정동별 내국인 유동인구수
cty_forn_02.head().sort_values(by='total', ascending=False)

# 시군구별 유입 생활이동인구

In [ ]:
# 시군구별 유입 생활이동인구
df_inflow = pd.read_csv('/content/drive/MyDrive/GG_TRANS_SEXAGE_SGG_INFLOW.csv')

# 부천시 시군구 경계가 업데이트 되지 않은 관계로 3개 시군구 통계내서 활용
df_inflow.loc[df_inflow['D_CTY_CD'].isin([41194, 41192, 41196]), 'D_CTY_CD'] = 41190
df_inflow.loc[df_inflow['D_CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'D_CTY_NM'] = '부천시'
df_inflow['D_CTY_CD'] = df_inflow['D_CTY_CD'].astype('str')

#opendw 데이터(시군구 경계)
cty_shp = pd.read_csv('/content/drive/MyDrive/시군구 경계/TBSHP_LH_CTY_202404081545.csv', encoding='UTF-8')
cty_shp['SGG_CD'] = cty_shp['SGG_CD'].astype('str')
cty_shp['GEOMETRY'] = cty_shp['GEOMETRY'].astype('str')
cty_shp['geometry'] = cty_shp['GEOMETRY'].apply(lambda wkt: loads(wkt))
cty_shp = gpd.GeoDataFrame(cty_shp, geometry = 'geometry').set_crs("EPSG:5181")
cty_shp = cty_shp.drop(columns = ['GEOMETRY'])
cty_shp = cty_shp[['SGG_CD', 'geometry']]
cty_shp = cty_shp.to_crs("EPSG:4326")

In [ ]:
df_inflow['year_mm'] = df_inflow['ETL_YM']
df_inflow.loc[df_inflow['D_CTY_CD'].isin([41194, 41192, 41196]), 'D_CTY_CD'] = 41190
df_inflow.loc[df_inflow['D_CTY_CD'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'CTY_NM'] = '부천시'

In [ ]:
grouped = df_inflow.groupby(['D_CTY_CD', 'D_CTY_NM', 'year_mm'])['CNT'].sum().reset_index()
grouped

In [ ]:
concat_data = pd.merge(grouped, cty_shp, left_on = 'D_CTY_CD', right_on = 'SGG_CD', how = 'left')
concat_data = gpd.GeoDataFrame(concat_data, geometry='geometry')
# concat_data

In [ ]:
cty_tmp_01 = concat_data[concat_data['year_mm'] == 202401]
map(cty_tmp_01, 'D_CTY_CD', 'CNT', '유입 생활이동인구', tootip = ['D_CTY_NM', 'CNT'])

In [ ]:
cty_tmp_02 = concat_data[concat_data['year_mm'] == 202402]
map(cty_tmp_02, 'D_CTY_CD', 'CNT', '유입 생활이동인구', tootip = ['D_CTY_NM', 'CNT'])

# 시군구별(TOP 5) 유입 이동수단별 생활이동인구

In [ ]:
sudan_mapping = {0: '차량',
                 1: '노선버스',
                 2: '지하철',
                 3: '도보',
                 4: '고속버스',
                 5: '기차',
                 6: '항공',
                 7: '기타'}

In [ ]:
sgg_sudan_income_data = pd.read_csv('./drive/MyDrive/GG_TRANS_SEXAGE_SGG_INFLOW.csv')
sgg_sudan_income_data['TRANS_GB'] = sgg_sudan_income_data['TRANS_GB'].map(sudan_mapping)

In [ ]:
year_mm_list = sgg_sudan_income_data['ETL_YM'].sort_values().drop_duplicates().to_list()

year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = sgg_sudan_income_data[sgg_sudan_income_data['ETL_YM'] == year_mm]
  temp = temp.groupby(by=['ETL_YM', 'D_CTY_NM'])['CNT'].sum().reset_index()
  temp = temp.sort_values(by='CNT', ascending=False)
  temp = temp[:5]
  temp_list = temp['D_CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list
# year_mm_si_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = sgg_sudan_income_data[sgg_sudan_income_data['ETL_YM'] == key]
  temp_df = temp_df[temp_df['D_CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])
# concat_data.head()

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'D_CTY_NM', 'TRANS_GB'])['CNT'].sum().reset_index()
group_data.head()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['D_CTY_NM', 'TRANS_GB'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['D_CTY_NM'] == si]
    create_barplot(visual_si_data, x='TRANS_GB', y='CNT', title=f'{key} {si}·이동수단별 유입 생활이동인구', xlabel='이동수단', ylabel='생활이동인구')

# 시군구별(TOP 5) 유입 이동목적별 생활이동인구

In [ ]:
mokjuk_mapping = {0: '귀가',
                 1: '출근',
                 2: '등교',
                 3: '쇼핑',
                 4: '관광',
                 5: '병원',
                 6: '기타'
                 }

In [ ]:
sgg_mokjuk_income_data = pd.read_csv('./drive/MyDrive/GG_PURPOSE_SEXAGE_SGG_INFLOW.csv')
sgg_mokjuk_income_data['PURPOSE'] = sgg_mokjuk_income_data['PURPOSE'].map(mokjuk_mapping)

In [ ]:
year_mm_list = sgg_mokjuk_income_data['ETL_YM'].sort_values().drop_duplicates().to_list()

year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = sgg_mokjuk_income_data[sgg_mokjuk_income_data['ETL_YM'] == year_mm]
  temp = temp.groupby(by=['ETL_YM', 'D_CTY_NM'])['CNT'].sum().reset_index()
  temp = temp.sort_values(by='CNT', ascending=False)
  temp = temp[:5]
  temp_list = temp['D_CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list
# year_mm_si_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = sgg_mokjuk_income_data[sgg_mokjuk_income_data['ETL_YM'] == key]
  temp_df = temp_df[temp_df['D_CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])
# concat_data.head()

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'D_CTY_NM', 'PURPOSE'])['CNT'].sum().reset_index()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['D_CTY_NM', 'PURPOSE'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])

sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['D_CTY_NM'] == si]
    create_barplot(visual_si_data, x='PURPOSE', y='CNT', title=f'{key} {si}·이동목적별 유입 생활이동인구', xlabel='이동목적', ylabel='생활이동인구')

# 시군구별(TOP 5) 유입 이동수단·목적 생활이동인구

In [ ]:
sgg_mokjuk_sudam_income_data = pd.read_csv('./drive/MyDrive/GG_PURPOSE_TRANS_SEXAGE_DURATION_ADMI_INFLOW.csv')

In [ ]:
sgg_mokjuk_sudam_income_data['sgg'] = sgg_mokjuk_sudam_income_data['D_ADMI_CD'].astype(str).str.slice(0, 5)

In [ ]:
sgg_mokjuk_sudam_income_group_data = sgg_mokjuk_sudam_income_data.groupby(["D_CTY_NM", "PURPOSE", "TRANS_GB", "sgg"])['CNT'].sum().reset_index()
sgg_mokjuk_sudam_income_group_data['PURPOSE'] = sgg_mokjuk_sudam_income_group_data['PURPOSE'].map(mokjuk_mapping)
sgg_mokjuk_sudam_income_group_data['TRANS_GB'] = sgg_mokjuk_sudam_income_group_data['TRANS_GB'].map(sudan_mapping)

In [ ]:
temp = sgg_mokjuk_sudam_income_group_data.groupby(['D_CTY_NM'])['CNT'].sum().reset_index()
temp = temp.sort_values(by='CNT', ascending=False)
temp = temp[:5]
top_5_list = temp['D_CTY_NM'].to_list()
top_5_list

In [ ]:
temp = sgg_mokjuk_sudam_income_group_data.copy()
sudan_list = sgg_mokjuk_sudam_income_group_data['TRANS_GB'].drop_duplicates().to_list()
mokjuk_list = sgg_mokjuk_sudam_income_group_data['PURPOSE'].drop_duplicates().to_list()
si_list = sgg_mokjuk_sudam_income_group_data['D_CTY_NM'].drop_duplicates().to_list()

print(len(sgg_mokjuk_sudam_income_group_data))

for si in si_list:
  for sudan in sudan_list:
    temp_data = sgg_mokjuk_sudam_income_group_data[(sgg_mokjuk_sudam_income_group_data['D_CTY_NM'] == si) & (sgg_mokjuk_sudam_income_group_data['TRANS_GB'] == sudan)]
    for mokjuk in mokjuk_list:
      if mokjuk not in temp_data['PURPOSE'].to_list():
        sgg_mokjuk_sudam_income_group_data = pd.concat([sgg_mokjuk_sudam_income_group_data,
                                                        pd.DataFrame({'D_CTY_NM': [si],
                                                                      'TRANS_GB': [sudan],
                                                                      'PURPOSE': [mokjuk],
                                                                      'CNT': [0]}
                                                                     )],ignore_index=True)

print(len(sgg_mokjuk_sudam_income_group_data))


In [ ]:
sgg_mokjuk_sudam_income_group_data

In [ ]:
for sigungu in top_5_list:
  top_data = sgg_mokjuk_sudam_income_group_data[sgg_mokjuk_sudam_income_group_data['D_CTY_NM'] == sigungu]
  sudan_group_data = top_data.groupby(['TRANS_GB'])['CNT'].sum().reset_index()
  sudan_mokjuk_group_data = top_data.groupby(['TRANS_GB', 'PURPOSE'])['CNT'].sum().reset_index()

  ax1 = sns.set_style(style=None, rc=None )

  fig, ax1 = plt.subplots(figsize=(12,6))
  sns.barplot(data = sudan_group_data, x='TRANS_GB', y='CNT', alpha=0.5, ax=ax1)

  # ax2 = ax1.twinx()
  sns.lineplot(data = sudan_mokjuk_group_data, x='TRANS_GB', y='CNT', hue='PURPOSE', marker='o', ax=ax1)

  # y축의 지수 제거
  ax1.ticklabel_format(axis='y',useOffset=False, style='plain')

  max_y = sudan_group_data['CNT'].max()  # y 열의 최댓값 계산

  plt.title(f'{sigungu}·이동수단·이동목적별 유입 생활이동인구', fontsize=16)
  plt.legend(loc='upper right')

  plt.ylim(0, max_y * 1.3)  # 최댓값의 30%를 여유 공간으로 추가
  plt.show()

# 시군구별(TOP 5) 유입 성·연령별 생활이동인구

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'D_CTY_NM', 'SEX_CD', 'AGE_GRP'])['CNT'].sum().reset_index()
group_data.head()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['D_CTY_NM', 'AGE_GRP', 'SEX_CD'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])

sort_df = sort_df[sort_df['AGE_GRP'] <= 7]
sort_df['AGE_GRP'] = sort_df['AGE_GRP']*10
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['D_CTY_NM'] == si]
    create_barplot(visual_si_data, x='AGE_GRP', y='CNT', title=f'{key} {si}·연령대별·성별 유입 생활이동인구', xlabel='연령대', ylabel='생활이동인구',  legend='SEX_CD')


# 시군구별(TOP 5) 유입 시간대별 생활이동인구

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'D_CTY_NM', 'SEX_CD', 'D_TIME_CD'])['CNT'].sum().reset_index()
group_data.head()

sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['D_CTY_NM', 'SEX_CD', 'D_TIME_CD'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['D_CTY_NM'] == si]
    create_lineplot(visual_si_data, x='D_TIME_CD', y='CNT', title=f'{key} {si}·시간대별·성별 유입 생활이동인구', xlabel='시간대', ylabel='생활이동인구',  legend='SEX_CD')

# 시군구별(TOP 5) 유입 방문목적·체류시간별 생활이동인구

In [ ]:
sgg_time_income_data = pd.read_csv('./drive/MyDrive/GG_PURPOSE_SEXAGE_DURATION_SGG_INFLOW.csv')

In [ ]:
sgg_time_income_data['DURATION'] = sgg_time_income_data['DURATION'].astype(float)
sgg_time_income_data['PURPOSE'] = sgg_time_income_data['PURPOSE'].map(mokjuk_mapping)
sgg_time_income_data

In [ ]:
year_mm_list = sgg_time_income_data['ETL_YM'].sort_values().drop_duplicates().to_list()
year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = sgg_time_income_data[sgg_time_income_data['ETL_YM'] == year_mm]
  temp = temp.groupby(by=['ETL_YM', 'D_CTY_NM'])['CNT'].sum().reset_index()
  temp = temp.sort_values(by='CNT', ascending=False)
  temp = temp[:5]
  temp_list = temp['D_CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list
# year_mm_si_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = sgg_time_income_data[sgg_time_income_data['ETL_YM'] == key]
  temp_df = temp_df[temp_df['D_CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])

concat_data['time'] = ''
concat_data.loc[concat_data['DURATION'] < 60, 'time'] = '1시간 미만'
concat_data.loc[(concat_data['DURATION'] >= 60) & (concat_data['DURATION'] < 120), 'time'] = '1시간 이상'
concat_data.loc[(concat_data['DURATION'] >= 120) & (concat_data['DURATION'] < 180), 'time'] = '2시간 이상'
concat_data.loc[concat_data['DURATION'] >= 180, 'time'] = '3시간 이상'
# concat_data.head()

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'D_CTY_NM', 'PURPOSE', 'time'])['CNT'].sum().reset_index()
# group_data

In [ ]:
sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['D_CTY_NM', 'time'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
purpose_list = sort_df['PURPOSE'].drop_duplicates().to_list()
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['D_CTY_NM'] == si]
    for purpose in purpose_list:
      visual_purpose_data = visual_si_data[visual_si_data['PURPOSE'] == purpose]
      check_list = visual_purpose_data['PURPOSE'].drop_duplicates().to_list()
      if purpose in check_list:
        create_barplot(visual_purpose_data, x='time', y='CNT', title=f'{key} {si} {purpose} 목적·체류시간별 유입 생활이동인구', xlabel='체류시간', ylabel='생활이동인구')

# 시군구별 유출 생활이동인구

In [ ]:
# 시군구별 유출 생활이동인구
df_outflow = pd.read_csv('/content/drive/MyDrive/GG_TRANS_SEXAGE_SGG_OUTFLOW.csv')

# 부천시 시군구 경계가 업데이트 되지 않은 관계로 3개 시군구 통계내서 활용
df_outflow.loc[df_outflow['O_CTY_CD'].isin([41194, 41192, 41196]), 'O_CTY_CD'] = 41190
df_outflow.loc[df_outflow['O_CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'O_CTY_NM'] = '부천시'
df_outflow['O_CTY_CD'] = df_outflow['O_CTY_CD'].astype('str')
df_outflow['year_mm'] = df_outflow['ETL_YM']

grouped = df_outflow.groupby(['O_CTY_CD', 'O_CTY_NM', 'year_mm'])['CNT'].sum().reset_index()

cty_outflow = pd.merge(grouped, cty_shp, left_on = 'O_CTY_CD', right_on = 'SGG_CD', how = 'left')
cty_outflow = gpd.GeoDataFrame(cty_outflow, geometry='geometry')
cty_outflow = cty_outflow[['year_mm', 'O_CTY_CD', 'O_CTY_NM', 'CNT', 'geometry']]

# 202401, 202402 총 유출 생활인구수 데이터
cty_tmp_03 = cty_outflow[cty_outflow['year_mm'] == 202401]
cty_tmp_03 = cty_tmp_03[['O_CTY_CD', 'O_CTY_NM', 'CNT', 'year_mm', 'geometry']]
cty_tmp_04 = cty_outflow[cty_outflow['year_mm'] == 202402]
cty_tmp_04 = cty_tmp_04[['O_CTY_CD', 'O_CTY_NM', 'CNT', 'year_mm', 'geometry']]

In [ ]:
map(cty_tmp_03, 'O_CTY_CD', 'CNT', '유출 생활이동인구', tootip = ['O_CTY_NM', 'CNT'])

In [ ]:
map(cty_tmp_04, 'O_CTY_CD', 'CNT', '유출 생활이동인구', tootip = ['O_CTY_NM', 'CNT'])

# 시군구별(TOP 5) 유출 이동수단별 생활이동인구

In [ ]:
sgg_sudan_outcome_data = pd.read_csv('./drive/MyDrive/GG_TRANS_SEXAGE_SGG_OUTFLOW.csv')
sgg_sudan_outcome_data['TRANS_GB'] = sgg_sudan_outcome_data['TRANS_GB'].map(sudan_mapping)
# sgg_sudan_outcome_data

In [ ]:
year_mm_list = sgg_sudan_outcome_data['ETL_YM'].sort_values().drop_duplicates().to_list()
year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = sgg_sudan_outcome_data[sgg_sudan_outcome_data['ETL_YM'] == year_mm]
  temp = temp.groupby(by=['ETL_YM', 'O_CTY_NM'])['CNT'].sum().reset_index()
  temp = temp.sort_values(by='CNT', ascending=False)
  temp = temp[:5]
  temp_list = temp['O_CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list
# year_mm_si_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = sgg_sudan_outcome_data[sgg_sudan_outcome_data['ETL_YM'] == key]
  temp_df = temp_df[temp_df['O_CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])
# concat_data.head()

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'O_CTY_NM', 'TRANS_GB'])['CNT'].sum().reset_index()
# group_data.head()

In [ ]:
sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['O_CTY_NM', 'TRANS_GB'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['O_CTY_NM'] == si]
    create_barplot(visual_si_data, x='TRANS_GB', y='CNT', title=f'{key} {si}·이동수단별 유출 생활이동인구', xlabel='이동수단', ylabel='생활이동인구')

# 시군구별(TOP 5) 유출 이동목적별 생활이동인구

In [ ]:
sgg_mokjuk_outcome_data = pd.read_csv('./drive/MyDrive/GG_PURPOSE_SEXAGE_SGG_OUTFLOW.csv')
sgg_mokjuk_outcome_data['PURPOSE'] = sgg_mokjuk_outcome_data['PURPOSE'].map(mokjuk_mapping)
sgg_mokjuk_outcome_data

In [ ]:
year_mm_list = sgg_mokjuk_outcome_data['ETL_YM'].sort_values().drop_duplicates().to_list()

year_mm_si_dic = {}
for year_mm in year_mm_list:
  temp = sgg_mokjuk_outcome_data[sgg_mokjuk_outcome_data['ETL_YM'] == year_mm]
  temp = temp.groupby(by=['ETL_YM', 'O_CTY_NM'])['CNT'].sum().reset_index()
  temp = temp.sort_values(by='CNT', ascending=False)
  temp = temp[:5]
  temp_list = temp['O_CTY_NM'].to_list()
  year_mm_si_dic[year_mm] = temp_list
# year_mm_si_dic

In [ ]:
concat_data = pd.DataFrame()
for key, si_list in year_mm_si_dic.items():
  temp_df = sgg_mokjuk_outcome_data[sgg_mokjuk_outcome_data['ETL_YM'] == key]
  temp_df = temp_df[temp_df['O_CTY_NM'].isin(si_list)]
  concat_data = pd.concat([concat_data, temp_df])
# concat_data.head()

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'O_CTY_NM', 'PURPOSE'])['CNT'].sum().reset_index()
# group_data.head()

In [ ]:
sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['O_CTY_NM', 'PURPOSE'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['O_CTY_NM'] == si]
    create_barplot(visual_si_data, x='PURPOSE', y='CNT', title=f'{key} {si}·이동목적별 유출 생활이동인구', xlabel='이동목적', ylabel='생활이동인구')

# 시군구별(TOP 5) 유출 이동수단·목적 생활이동인구

In [ ]:
sgg_mokjuk_sudam_outcome_data = pd.read_csv('./drive/MyDrive/GG_PURPOSE_TRANS_SEXAGE_DURATION_ADMI_OUTFLOW.csv')
sgg_mokjuk_sudam_outcome_data['sgg'] = sgg_mokjuk_sudam_outcome_data['O_ADMI_CD'].astype(str).str.slice(0, 5)

In [ ]:
sgg_mokjuk_sudam_outcome_group_data = sgg_mokjuk_sudam_outcome_data.groupby(["O_CTY_NM", "PURPOSE", "TRANS_GB", "sgg"])['CNT'].sum().reset_index()
sgg_mokjuk_sudam_outcome_group_data['PURPOSE'] = sgg_mokjuk_sudam_outcome_group_data['PURPOSE'].map(mokjuk_mapping)
sgg_mokjuk_sudam_outcome_group_data['TRANS_GB'] = sgg_mokjuk_sudam_outcome_group_data['TRANS_GB'].map(sudan_mapping)
sgg_mokjuk_sudam_outcome_group_data

In [ ]:
temp = sgg_mokjuk_sudam_outcome_group_data.groupby(['O_CTY_NM'])['CNT'].sum().reset_index()
temp = temp.sort_values(by='CNT', ascending=False)
temp = temp[:5]
top_5_list = temp['O_CTY_NM'].to_list()
top_5_list

In [ ]:
temp = sgg_mokjuk_sudam_outcome_group_data.copy()
sudan_list = sgg_mokjuk_sudam_outcome_group_data['TRANS_GB'].drop_duplicates().to_list()
mokjuk_list = sgg_mokjuk_sudam_outcome_group_data['PURPOSE'].drop_duplicates().to_list()
si_list = sgg_mokjuk_sudam_outcome_group_data['O_CTY_NM'].drop_duplicates().to_list()

print(len(sgg_mokjuk_sudam_outcome_group_data))

for si in si_list:
  for sudan in sudan_list:
    temp_data = sgg_mokjuk_sudam_outcome_group_data[(sgg_mokjuk_sudam_outcome_group_data['O_CTY_NM'] == si) & (sgg_mokjuk_sudam_outcome_group_data['TRANS_GB'] == sudan)]
    for mokjuk in mokjuk_list:
      if mokjuk not in temp_data['PURPOSE'].to_list():
        sgg_mokjuk_sudam_outcome_group_data = pd.concat([sgg_mokjuk_sudam_outcome_group_data,
                                                        pd.DataFrame({'O_CTY_NM': [si],
                                                                      'TRANS_GB': [sudan],
                                                                      'PURPOSE': [mokjuk],
                                                                      'CNT': [0]}
                                                                     )],ignore_index=True)

print(len(sgg_mokjuk_sudam_outcome_group_data))


In [ ]:
for sigungu in top_5_list:
  top_data = sgg_mokjuk_sudam_outcome_group_data[sgg_mokjuk_sudam_outcome_group_data['O_CTY_NM'] == sigungu]
  sudan_group_data = top_data.groupby(['TRANS_GB'])['CNT'].sum().reset_index()
  sudan_mokjuk_group_data = top_data.groupby(['TRANS_GB', 'PURPOSE'])['CNT'].sum().reset_index()

  ax1 = sns.set_style(style=None, rc=None )

  fig, ax1 = plt.subplots(figsize=(12,6))
  sns.barplot(data = sudan_group_data, x='TRANS_GB', y='CNT', alpha=0.5, ax=ax1)

  # ax2 = ax1.twinx()
  sns.lineplot(data = sudan_mokjuk_group_data, x='TRANS_GB', y='CNT', hue='PURPOSE', marker='o', ax=ax1)

  # y축의 지수 제거
  ax1.ticklabel_format(axis='y',useOffset=False, style='plain')
  # ax1.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
  # ax2.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))

  max_1 = sudan_group_data['CNT'].max()  # y 열의 최댓값 계산
  max_2 = sudan_mokjuk_group_data['CNT'].max()  # y 열의 최댓값 계산

  max_y = max(max_1, max_2)

  plt.title(f'{sigungu}·이동수단·이동목적별 유출 생활이동인구', fontsize=16)
  plt.legend(loc='upper right')

  # plt.ylim(0, max_y * 1.3)  # 최댓값의 30%를 여유 공간으로 추가
  plt.show()

# 시군구별(TOP 5) 유출 성·연령별 생활이동인구

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'O_CTY_NM', 'SEX_CD', 'AGE_GRP'])['CNT'].sum().reset_index()
# group_data.head()

In [ ]:
sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['O_CTY_NM', 'AGE_GRP', 'SEX_CD'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])

sort_df = sort_df[sort_df['AGE_GRP'] <= 7]
sort_df['AGE_GRP'] = sort_df['AGE_GRP']*10
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['O_CTY_NM'] == si]
    create_barplot(visual_si_data, x='AGE_GRP', y='CNT', title=f'{key} {si}·연령대별·성별 유출 생활이동인구', xlabel='연령대', ylabel='생활이동인구',  legend='SEX_CD')

# 시군구별(TOP 5) 유출 시간대별 생활이동인구

In [ ]:
group_data = concat_data.groupby(['ETL_YM', 'O_CTY_NM', 'SEX_CD', 'O_TIME_CD'])['CNT'].sum().reset_index()
# group_data.head()

In [ ]:
sort_df = pd.DataFrame()
for key_2, si_list in year_mm_si_dic.items():
  temp = group_data[group_data['ETL_YM'] == key_2]
  temp = temp.sort_values(by=['O_CTY_NM', 'SEX_CD', 'O_TIME_CD'], key=lambda x: x.map({v: i for i, v in enumerate(si_list)}))
  sort_df = pd.concat([sort_df, temp])
sort_df

In [ ]:
for key, si_list in year_mm_si_dic.items():
  visual_data = sort_df[sort_df['ETL_YM'] == key]
  for si in si_list:
    visual_si_data = visual_data[visual_data['O_CTY_NM'] == si]
    create_lineplot(visual_si_data, x='O_TIME_CD', y='CNT', title=f'{key} {si}·시간대별·성별 유출 생활이동인구', xlabel='시간대', ylabel='생활이동인구',  legend='SEX_CD')

# 수원시 영통구(경기도청)의 유입/유출 등시선 지도

## 시군구

### 유입

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/GG_PURPOSE_TRANS_SEXAGE_SGG_OD.csv')
data

In [ ]:
data.loc[data['O_CTY_CD'].isin([41194, 41192, 41196]), 'O_CTY_CD'] = 41190
data.loc[data['O_CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'O_CTY_NM'] = '부천시'


data.loc[data['D_CTY_CD'].isin([41194, 41192, 41196]), 'D_CTY_CD'] = 41190
data.loc[data['D_CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'D_CTY_NM'] = '부천시'

In [ ]:
data['O_CTY_CD'] = data['O_CTY_CD'].astype(str).str.split('.').str[0]
data['D_CTY_CD'] = data['D_CTY_CD'].astype(str).str.split('.').str[0]

In [ ]:
suwon_data = data[data['D_CTY_NM'] == '수원시 영통구']

In [ ]:
group_suwon_data = suwon_data.groupby(by=['O_CTY_CD', 'O_CTY_NM'])['CNT'].sum().reset_index()
group_suwon_data = group_suwon_data.sort_values(by='CNT', ascending=False)
group_suwon_data = group_suwon_data[group_suwon_data['O_CTY_NM'] != '수원시 영통구']

In [ ]:
cty_shp

In [ ]:
cty_tmp = pd.merge(group_suwon_data, cty_shp, left_on = 'O_CTY_CD', right_on = 'SGG_CD', how = 'left')
cty_tmp = gpd.GeoDataFrame(cty_tmp, geometry='geometry')
cty_tmp['o_sido'] = cty_tmp['O_CTY_CD'].str.slice(0, 2)
cty_tmp = cty_tmp[cty_tmp['o_sido'] == '41']

In [ ]:
quantiles_column_A = cty_tmp['CNT'].quantile([0.25, 0.5, 0.75])
# quantiles_column_A = cty_tmp['CNT'].quantile([0.2, 0.4, 0.6, 0.8])
print(quantiles_column_A)
# print(quantiles_column_A[0.25])

In [ ]:
cty_tmp['value'] = ''
cty_tmp.loc[cty_tmp['CNT']<= quantiles_column_A[0.25], 'value'] = 0.25
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.25]) & (cty_tmp['CNT'] <= quantiles_column_A[0.5]), 'value'] = 0.5
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.5]) & (cty_tmp['CNT'] <= quantiles_column_A[0.75]), 'value'] = 0.75
cty_tmp.loc[cty_tmp['CNT'] > quantiles_column_A[0.75], 'value'] = 1

# 추후에 행정동 부분에서 사용
in_many_sgg_list = cty_tmp.loc[cty_tmp['value'] == 1, 'O_CTY_NM'].to_list()
in_many_sgg_list

In [ ]:
cty_tmp.explore(column="value", categorical=True, cmap='YlOrRd')

### 유출

In [ ]:
suwon_data_out = data[data['O_CTY_NM'] == '수원시 영통구']

In [ ]:
group_suwon_out_data = suwon_data_out.groupby(by=['D_CTY_CD', 'D_CTY_NM'])['CNT'].sum().reset_index()
group_suwon_out_data = group_suwon_out_data.sort_values(by='CNT', ascending=False)
group_suwon_out_data = group_suwon_out_data[group_suwon_out_data['D_CTY_NM'] != '수원시 영통구']

In [ ]:
cty_tmp = pd.merge(group_suwon_out_data, cty_shp, left_on = 'D_CTY_CD', right_on = 'SGG_CD', how = 'left')
cty_tmp = gpd.GeoDataFrame(cty_tmp, geometry='geometry')
cty_tmp['d_sido'] = cty_tmp['D_CTY_CD'].str.slice(0, 2)
cty_tmp = cty_tmp[cty_tmp['d_sido'] == '41']

In [ ]:
quantiles_column_A = cty_tmp['CNT'].quantile([0.25, 0.5, 0.75])
print(quantiles_column_A)

In [ ]:
cty_tmp['value'] = ''
cty_tmp.loc[cty_tmp['CNT']<= quantiles_column_A[0.25], 'value'] = 0.25
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.25]) & (cty_tmp['CNT'] <= quantiles_column_A[0.5]), 'value'] = 0.5
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.5]) & (cty_tmp['CNT'] <= quantiles_column_A[0.75]), 'value'] = 0.75
cty_tmp.loc[cty_tmp['CNT'] > quantiles_column_A[0.75], 'value'] = 1

out_many_sgg_list = cty_tmp.loc[cty_tmp['value'] == 1, 'D_CTY_NM'].to_list()
out_many_sgg_list

In [ ]:
cty_tmp.explore(column="value", categorical=True, cmap='YlOrRd')

### 유입/유출이 가장 많이 일어난 시군구 확인(Top 5)

In [ ]:
group_data = data.groupby(['O_CTY_CD', 'D_CTY_CD', 'O_CTY_NM', 'D_CTY_NM'])['CNT'].sum().reset_index()
group_data = group_data.sort_values(by='CNT', ascending=False)
group_data = group_data[group_data['O_CTY_NM'] != group_data['D_CTY_NM']]

In [ ]:
cty_tmp = pd.merge(group_data, cty_shp, left_on = 'O_CTY_CD', right_on = 'SGG_CD', how = 'left')
change_column_names = {'geometry': 'o_geometry'}
cty_tmp = cty_tmp.rename(columns=change_column_names)

In [ ]:
cty_tmp = pd.merge(cty_tmp, cty_shp, left_on = 'D_CTY_CD', right_on = 'SGG_CD', how = 'left')
change_column_names = {'geometry': 'd_geometry'}
cty_tmp = cty_tmp.rename(columns=change_column_names)
top_5 = cty_tmp[:5]

In [ ]:
top_5

In [ ]:
top_1 = 0

visual_o = top_5[['O_CTY_CD', 'O_CTY_NM', 'o_geometry', 'CNT']]
change_column_names = {'o_geometry': 'geometry'}
visual_o = visual_o.rename(columns=change_column_names)
visual_o = visual_o.iloc[[top_1]]
visual_o = gpd.GeoDataFrame(visual_o, geometry='geometry')

visual_d = top_5[['D_CTY_CD', 'D_CTY_NM', 'd_geometry', 'CNT']]
change_column_names = {'d_geometry': 'geometry'}
visual_d = visual_d.rename(columns=change_column_names)
visual_d = visual_d.iloc[[top_1]]
visual_d = gpd.GeoDataFrame(visual_d, geometry='geometry')

map_3(visual_o, visual_d)

In [ ]:
top_1 = 1

visual_o = top_5[['O_CTY_CD', 'O_CTY_NM', 'o_geometry', 'CNT']]
change_column_names = {'o_geometry': 'geometry'}
visual_o = visual_o.rename(columns=change_column_names)
visual_o = visual_o.iloc[[top_1]]
visual_o = gpd.GeoDataFrame(visual_o, geometry='geometry')

visual_d = top_5[['D_CTY_CD', 'D_CTY_NM', 'd_geometry', 'CNT']]
change_column_names = {'d_geometry': 'geometry'}
visual_d = visual_d.rename(columns=change_column_names)
visual_d = visual_d.iloc[[top_1]]
visual_d = gpd.GeoDataFrame(visual_d, geometry='geometry')

map_3(visual_o, visual_d)

In [ ]:
top_1 = 2

visual_o = top_5[['O_CTY_CD', 'O_CTY_NM', 'o_geometry', 'CNT']]
change_column_names = {'o_geometry': 'geometry'}
visual_o = visual_o.rename(columns=change_column_names)
visual_o = visual_o.iloc[[top_1]]
visual_o = gpd.GeoDataFrame(visual_o, geometry='geometry')

visual_d = top_5[['D_CTY_CD', 'D_CTY_NM', 'd_geometry', 'CNT']]
change_column_names = {'d_geometry': 'geometry'}
visual_d = visual_d.rename(columns=change_column_names)
visual_d = visual_d.iloc[[top_1]]
visual_d = gpd.GeoDataFrame(visual_d, geometry='geometry')

map_3(visual_o, visual_d)

In [ ]:
top_1 = 3

visual_o = top_5[['O_CTY_CD', 'O_CTY_NM', 'o_geometry', 'CNT']]
change_column_names = {'o_geometry': 'geometry'}
visual_o = visual_o.rename(columns=change_column_names)
visual_o = visual_o.iloc[[top_1]]
visual_o = gpd.GeoDataFrame(visual_o, geometry='geometry')

visual_d = top_5[['D_CTY_CD', 'D_CTY_NM', 'd_geometry', 'CNT']]
change_column_names = {'d_geometry': 'geometry'}
visual_d = visual_d.rename(columns=change_column_names)
visual_d = visual_d.iloc[[top_1]]
visual_d = gpd.GeoDataFrame(visual_d, geometry='geometry')

map_3(visual_o, visual_d)

In [ ]:
top_1 = 4

visual_o = top_5[['O_CTY_CD', 'O_CTY_NM', 'o_geometry', 'CNT']]
change_column_names = {'o_geometry': 'geometry'}
visual_o = visual_o.rename(columns=change_column_names)
visual_o = visual_o.iloc[[top_1]]
visual_o = gpd.GeoDataFrame(visual_o, geometry='geometry')

visual_d = top_5[['D_CTY_CD', 'D_CTY_NM', 'd_geometry', 'CNT']]
change_column_names = {'d_geometry': 'geometry'}
visual_d = visual_d.rename(columns=change_column_names)
visual_d = visual_d.iloc[[top_1]]
visual_d = gpd.GeoDataFrame(visual_d, geometry='geometry')

map_3(visual_o, visual_d)

## 행정동

### 유입

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/GG_TRANS_SEXAGE_ADMI_OD.csv')

In [ ]:
from pyproj import CRS

katech_proj_string = """
    +proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000
    +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43
    """

# Proj 문자열을 CRS 객체로 변환
katech_crs = CRS.from_proj4(katech_proj_string)

cty_shp = pd.read_csv('/content/drive/MyDrive/행정동 경계/TBSHP_ADMI_20240104_202404090850.csv', encoding='UTF-8')
cty_shp['ADMI_CD'] = cty_shp['ADMI_CD'].astype('str')
cty_shp['GEOMETRY'] = cty_shp['GEOMETRY'].astype('str')
cty_shp['geometry'] = cty_shp['GEOMETRY'].apply(lambda wkt: loads(wkt))
# cty_shp = gpd.GeoDataFrame(cty_shp, geometry = 'geometry').set_crs("EPSG:5181")
cty_shp = gpd.GeoDataFrame(cty_shp, geometry = 'geometry').set_crs(katech_crs)
cty_shp = cty_shp.drop(columns = ['GEOMETRY'])
cty_shp = cty_shp[['ADMI_CD', 'geometry']]
cty_shp = cty_shp.to_crs("EPSG:4326")
cty_shp

In [ ]:
# data.loc[data['O_CTY_CD'].isin([41194, 41192, 41196]), 'O_CTY_CD'] = 41190
data.loc[data['O_CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'O_CTY_NM'] = '부천시'

In [ ]:
many_in_data = data[data['O_CTY_NM'].isin(in_many_sgg_list)]
many_in_data = many_in_data[(many_in_data['O_MEGA_NM'] == '경기도') & (many_in_data['D_MEGA_NM'] == '경기도')]

In [ ]:
many_in_group_data = many_in_data[(many_in_data['D_CTY_NM'] == '수원시 영통구') & (many_in_data['D_ADMI_NM'] == '광교1동')]
many_in_group_data = many_in_group_data.groupby(["O_ADMI_CD", "O_CTY_NM", "O_ADMI_NM"])['CNT'].sum().reset_index()
many_in_group_data['O_ADMI_CD'] = many_in_group_data['O_ADMI_CD'].astype(str).str.split('.').str[0]
many_in_group_data = many_in_group_data.sort_values(by='CNT', ascending=False)
many_in_group_data['admi'] = many_in_group_data['O_CTY_NM'] + ' ' + many_in_group_data['O_ADMI_NM']
many_in_group_data

In [ ]:
cty_tmp = pd.merge(many_in_group_data, cty_shp, left_on = 'O_ADMI_CD', right_on = 'ADMI_CD', how = 'left')
cty_tmp = gpd.GeoDataFrame(cty_tmp, geometry='geometry')
cty_tmp

In [ ]:
quantiles_column_A = cty_tmp['CNT'].quantile([0.25, 0.5, 0.75])
print(quantiles_column_A)

In [ ]:
cty_tmp['value'] = ''
cty_tmp.loc[cty_tmp['CNT']<= quantiles_column_A[0.25], 'value'] = 0.25
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.25]) & (cty_tmp['CNT'] <= quantiles_column_A[0.5]), 'value'] = 0.5
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.5]) & (cty_tmp['CNT'] <= quantiles_column_A[0.75]), 'value'] = 0.75
cty_tmp.loc[cty_tmp['CNT'] > quantiles_column_A[0.75], 'value'] = 1

In [ ]:
cty_tmp.explore(column="value", categorical=True, cmap='YlOrRd')

### 유출

In [ ]:
many_out_data = data[data['D_CTY_NM'].isin(out_many_sgg_list)]
many_out_data = many_out_data[(many_out_data['O_MEGA_NM'] == '경기도') & (many_out_data['D_MEGA_NM'] == '경기도')]
many_out_data

In [ ]:
many_out_group_data = many_out_data[(many_out_data['O_CTY_NM'] == '수원시 영통구') & (many_out_data['O_ADMI_NM'] == '광교1동')]
many_out_group_data = many_out_group_data.groupby(["D_ADMI_CD", "D_CTY_NM", "D_ADMI_NM"])['CNT'].sum().reset_index()
many_out_group_data['D_ADMI_CD'] = many_out_group_data['D_ADMI_CD'].astype(str).str.split('.').str[0]
many_out_group_data = many_out_group_data.sort_values(by='CNT', ascending=False)
many_out_group_data['admi'] = many_out_group_data['D_CTY_NM'] + ' ' + many_out_group_data['D_ADMI_NM']
many_out_group_data

In [ ]:
cty_tmp = pd.merge(many_out_group_data, cty_shp, left_on = 'D_ADMI_CD', right_on = 'ADMI_CD', how = 'left')
cty_tmp = gpd.GeoDataFrame(cty_tmp, geometry='geometry')
cty_tmp

In [ ]:
quantiles_column_A = cty_tmp['CNT'].quantile([0.25, 0.5, 0.75])
print(quantiles_column_A)

In [ ]:
cty_tmp['value'] = ''
cty_tmp.loc[cty_tmp['CNT']<= quantiles_column_A[0.25], 'value'] = 0.25
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.25]) & (cty_tmp['CNT'] <= quantiles_column_A[0.5]), 'value'] = 0.5
cty_tmp.loc[(cty_tmp['CNT'] > quantiles_column_A[0.5]) & (cty_tmp['CNT'] <= quantiles_column_A[0.75]), 'value'] = 0.75
cty_tmp.loc[cty_tmp['CNT'] > quantiles_column_A[0.75], 'value'] = 1

In [ ]:
cty_tmp.explore(column="value", categorical=True, cmap='YlOrRd')

# 시군구별(TOP 5) 요일별 탄소배출량

In [ ]:
carbon_data = pd.read_csv('./drive/MyDrive/GG_TRANS_CARBON_SGG_OUTFLOW.csv')

In [ ]:
sudan_mapping = {0: '차량',
                 1: '노선버스',
                 2: '지하철',
                 3: '도보',
                 4: '고속버스',
                 5: '기차',
                 6: '항공',
                 7: '기타'}
carbon_data['TRANS_GB'] = carbon_data['TRANS_GB'].map(sudan_mapping)
carbon_data

In [ ]:
group = carbon_data[(carbon_data['O_CTY_NM'] == '포천시') & (carbon_data['TRANS_GB'] == '차량')]
group.reset_index(drop = True)

In [ ]:
grouped = group.groupby('O_CTY_NM')['TRANS_GB'].count().reset_index() #.unstack(fill_value=0)
grouped

In [ ]:
carbon_data.loc[carbon_data['O_CTY_CD'].isin([41194, 41192, 41196]), 'O_CTY_CD'] = 41190
carbon_data.loc[carbon_data['O_CTY_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'O_CTY_NM'] = '부천시'
carbon_data['O_CTY_CD'] = carbon_data['O_CTY_CD'].astype('str')

In [ ]:
carbon_data['sort_value'] = ''
carbon_data.loc[carbon_data['DOW'] == '월', 'sort_value'] = 1
carbon_data.loc[carbon_data['DOW'] == '화', 'sort_value'] = 2
carbon_data.loc[carbon_data['DOW'] == '수', 'sort_value'] = 3
carbon_data.loc[carbon_data['DOW'] == '목', 'sort_value'] = 4
carbon_data.loc[carbon_data['DOW'] == '금', 'sort_value'] = 5
carbon_data.loc[carbon_data['DOW'] == '토', 'sort_value'] = 6
carbon_data.loc[carbon_data['DOW'] == '일', 'sort_value'] = 7
# carbon_data.tail()

In [ ]:
carbon_group = carbon_data.groupby(['ETL_YM', 'O_CTY_NM', 'DOW', 'sort_value'])['CARBON_EMISSIONS'].sum().reset_index()
carbon_group = carbon_group.sort_values(by='sort_value')
# carbon_group.head()

In [ ]:
found_top5 = carbon_data.groupby(['O_CTY_NM'])['CARBON_EMISSIONS'].sum().reset_index()
found_top5 = found_top5.sort_values(by=['CARBON_EMISSIONS'], ascending=False)
found_top5 = found_top5[:5]
found_top5_si_list = found_top5['O_CTY_NM'].to_list()
carbon_top5_data = carbon_group[carbon_group['O_CTY_NM'].isin(found_top5_si_list)]
date_list = carbon_top5_data['ETL_YM'].drop_duplicates().sort_values().to_list()

In [ ]:
for date in date_list:
  carbon_date_data = carbon_top5_data[carbon_top5_data['ETL_YM'] == date]
  for si in found_top5_si_list:
    carbon_group_si = carbon_date_data[carbon_date_data['O_CTY_NM'] == si]
    create_lineplot(carbon_group_si, x='DOW', y='CARBON_EMISSIONS', title=f'{date} {si} 요일별 탄소배출량', xlabel='요일', ylabel='탄소배출량')

# 시군구별 요일별 탄소배출량 지도시각화

In [ ]:
cty_shp = pd.read_csv('/content/drive/MyDrive/시군구 경계/TBSHP_LH_CTY_202404081545.csv', encoding='UTF-8')
cty_shp['SGG_CD'] = cty_shp['SGG_CD'].astype('str')
cty_shp['GEOMETRY'] = cty_shp['GEOMETRY'].astype('str')
cty_shp['geometry'] = cty_shp['GEOMETRY'].apply(lambda wkt: loads(wkt))
cty_shp = gpd.GeoDataFrame(cty_shp, geometry = 'geometry').set_crs("EPSG:5181")
cty_shp = cty_shp.drop(columns = ['GEOMETRY'])
cty_shp = cty_shp[['SGG_CD', 'geometry']]
cty_shp = cty_shp.to_crs("EPSG:4326")

In [ ]:
group_data = carbon_data.groupby(by=['O_CTY_CD', 'O_CTY_NM', 'DOW'])['CARBON_EMISSIONS'].sum().reset_index()
group_data = group_data.sort_values(by='CARBON_EMISSIONS', ascending=False)
group_data['O_CTY_CD'] = group_data['O_CTY_CD'].astype(str)

In [ ]:
cty_merge = pd.merge(group_data, cty_shp, left_on = 'O_CTY_CD', right_on = 'SGG_CD', how = 'left')
cty_merge = gpd.GeoDataFrame(cty_merge, geometry='geometry')
cty_merge

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '월']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='월요일 탄소배출량')

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '화']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='화요일 탄소배출량')

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '수']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='수요일 탄소배출량')

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '목']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='목요일 탄소배출량')

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '금']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='금요일 탄소배출량')

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '토']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='토요일 탄소배출량')

In [ ]:
week_data = cty_merge[cty_merge['DOW'] == '일']
map(week_data, 'O_CTY_NM', 'CARBON_EMISSIONS', legend_name='일요일 탄소배출량')

# 카드매출

## 시간대별 카드사 업종별 매출액 평균

In [ ]:
sale_df = pd.read_csv('/content/drive/MyDrive/data/nice데이터/카드데이터/cluster_gyeonggi_agrde_2401.csv', encoding = 'euc-kr') # 성,연령별 카드매출 데이터 load
cty_shp = pd.read_csv('/content/drive/MyDrive/시군구 경계/TBSHP_LH_CTY_202404081545.csv') # 시군구 경계 load
admi_shp = pd.read_csv('/content/drive/MyDrive/행정동 경계/TBSHP_ADMI_20240104_202405030948.csv') # 행정동 경계 load
top = 3 # 상위 3개

In [ ]:
time_dic = {
	1 : "00:00 ~ 06:59",
	2 : "07:00 ~ 08:59",
	3 : "09:00 ~ 10:59",
	4 : "11:00 ~ 12:59",
	5 : "13:00 ~ 14:59",
	6 : "15:00 ~ 16:59",
	7 : "17:00 ~ 18:59",
	8 : "19:00 ~ 20:59",
	9 : "21:00 ~ 22:59",
	10 : "23:00~ 23:59"
}

# 행정동별 시간대별 카드사 소분류 업종별 매출액 평균
admi_sales = sale_df.groupby(['admi_cty_no', 'hour', 'card_tpbuz_nm_3'])['amt'].mean().reset_index()
admi_sh_sales = pd.merge(admi_sales, admi_shp, left_on='admi_cty_no', right_on='ADMI_CD', how='left')
admi_sh_sales['hour2'] = admi_sh_sales['hour'].map(time_dic)

# 행정동별 시간대별 카드사 중분류 업종별 매출액 평균
admi_sales = sale_df.groupby(['admi_cty_no', 'hour', 'card_tpbuz_nm_2'])['amt'].mean().reset_index()
admi_mh_sales = pd.merge(admi_sales, admi_shp, left_on='admi_cty_no', right_on='ADMI_CD', how='left')
admi_mh_sales['hour2'] = admi_mh_sales['hour'].map(time_dic)

# 행정동별 시간대별 카드사 대분류 업종별 매출액 평균
admi_sales = sale_df.groupby(['admi_cty_no', 'hour', 'card_tpbuz_nm_1'])['amt'].mean().reset_index()
admi_lh_sales = pd.merge(admi_sales, admi_shp, left_on='admi_cty_no', right_on='ADMI_CD', how='left')
admi_lh_sales['hour2'] = admi_lh_sales['hour'].map(time_dic)

# 시군구별 시간대별 카드사 소분류 업종별 매출액 평균
cty_sales = sale_df.groupby(['cty_rgn_no', 'hour', 'card_tpbuz_nm_3'])['amt'].mean().reset_index()
cty_sh_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')
cty_sh_sales['hour2'] = cty_sh_sales['hour'].map(time_dic)

# 시군구별 시간대별 카드사 중분류 업종별 매출액 평균
cty_sales = sale_df.groupby(['cty_rgn_no', 'hour', 'card_tpbuz_nm_2'])['amt'].mean().reset_index()
cty_mh_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')
cty_mh_sales['hour2'] = cty_mh_sales['hour'].map(time_dic)

# 시군구별 시간대별 카드사 대분류 업종별 매출액 평균
cty_sales = sale_df.groupby(['cty_rgn_no', 'hour', 'card_tpbuz_nm_1'])['amt'].mean().reset_index()
cty_lh_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD')
cty_lh_sales['hour2'] = cty_lh_sales['hour'].map(time_dic)

# 가장 많은 상위 3개 카테고리
top_cate_1 = cty_lh_sales.groupby('card_tpbuz_nm_1')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_1'].to_list()[:top]
top_cate_2 = cty_mh_sales.groupby('card_tpbuz_nm_2')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_2'].to_list()[:top]
top_cate_3 = cty_sh_sales.groupby('card_tpbuz_nm_3')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_3'].to_list()[:top]
top_sido = cty_lh_sales.groupby('SGG_NM')['amt'].sum().sort_values(ascending=False).reset_index()['SGG_NM'].to_list()[:top]


In [ ]:
for sido in top_sido:
  visual_data = cty_lh_sales[cty_lh_sales['SGG_NM'] == sido]
  visual_data = visual_data[visual_data['card_tpbuz_nm_1'].isin(top_cate_1)]
  create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_1')

In [ ]:
for sido in top_sido:
  visual_data = cty_mh_sales[cty_mh_sales['SGG_NM'] == sido]
  visual_data = visual_data[visual_data['card_tpbuz_nm_2'].isin(top_cate_2)]
  create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_2')

In [ ]:
for sido in top_sido:
  visual_data = cty_sh_sales[cty_sh_sales['SGG_NM'] == sido]
  visual_data = visual_data[visual_data['card_tpbuz_nm_3'].isin(top_cate_3)]
  create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_3')

## 시군구별 카드사 업종별 매출액 평균

In [ ]:
chunk_size = 1000000
chunks = []

use_col = ['cty_rgn_no', 'admi_cty_no', 'card_tpbuz_cd', 'card_tpbuz_nm_1', 'card_tpbuz_nm_2', 'card_tpbuz_nm_3', 'inflow_cd', 'sex', 'age', 'day', 'amt', 'cnt']

for chunk in pd.read_csv('/content/drive/MyDrive/data/nice데이터/카드데이터/tbsh_gyeonggi_day_inflow_2401.csv',
                         encoding = 'euc-kr',
                         usecols = use_col,
                         chunksize=chunk_size):
    chunks.append(chunk)

inflow_df = pd.concat(chunks, ignore_index=True) # 유입지별 매출 데이터 load
top = 1 # 상위 1개

In [ ]:
inflow_df.loc[inflow_df['cty_rgn_no'].isin([41194, 41192, 41196]), 'cty_rgn_no'] = 41190

# 행정동별 카드사 소분류 업종별 매출액 평균
admi_sales = inflow_df.groupby(['admi_cty_no', 'card_tpbuz_cd', 'card_tpbuz_nm_3'])['amt'].mean().reset_index()
admi_s_sales = pd.merge(admi_sales, admi_shp, left_on='admi_cty_no', right_on='ADMI_CD', how='left')

# 행정동별 카드사 중분류 업종별 매출액 평균
admi_sales = inflow_df.groupby(['admi_cty_no', 'card_tpbuz_cd', 'card_tpbuz_nm_2'])['amt'].mean().reset_index()
admi_m_sales = pd.merge(admi_sales, admi_shp, left_on='admi_cty_no', right_on='ADMI_CD', how='left')

# 행정동별 카드사 대분류 업종별 매출액 평균
admi_sales = inflow_df.groupby(['admi_cty_no', 'card_tpbuz_cd', 'card_tpbuz_nm_1'])['amt'].mean().reset_index()
admi_L_sales = pd.merge(admi_sales, admi_shp, left_on='admi_cty_no', right_on='ADMI_CD', how='left')

# 시군구별 카드사 소분류 업종별 매출액 평균
cty_sales = inflow_df.groupby(['cty_rgn_no', 'card_tpbuz_cd', 'card_tpbuz_nm_3'])['amt'].mean().reset_index()
cty_s_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

# 시군구별 카드사 중분류 업종별 매출액 평균
cty_sales = inflow_df.groupby(['cty_rgn_no', 'card_tpbuz_nm_2'])['amt'].mean().reset_index()
cty_m_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

# 시군구별 카드사 대분류 업종별 매출액 평균
cty_sales = inflow_df.groupby(['cty_rgn_no',  'card_tpbuz_nm_1'])['amt'].mean().reset_index()
cty_L_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

In [ ]:
# 가장 많은 상위 1개 카테고리
top_cate_1 = cty_L_sales.groupby('card_tpbuz_nm_1')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_1'].to_list()[:top]
top_cate_2 = cty_m_sales.groupby('card_tpbuz_nm_2')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_2'].to_list()[:top]
top_cate_3 = cty_s_sales.groupby('card_tpbuz_nm_3')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_3'].to_list()[:top]
top_sido = cty_L_sales.groupby('SGG_NM')['amt'].sum().sort_values(ascending=False).reset_index()['SGG_NM'].to_list()[:top]

## 시군구별 가장 많은 업종(대,중,소) 평균 매출액

In [ ]:
# visual_data = cty_L_sales[cty_L_sales['SGG_NM'] == top_sido[0]]
visual_data = cty_L_sales[cty_L_sales['card_tpbuz_nm_1'].isin(top_cate_1)]


visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'CTY_NM'] = '부천시'
visual_data['geometry'] = visual_data['GEOMETRY'].apply(lambda wkt: loads(wkt))
visual_data = gpd.GeoDataFrame(visual_data, geometry = 'geometry').set_crs("EPSG:5181")
visual_data = visual_data.drop(columns = ['GEOMETRY'])
visual_data = visual_data.to_crs("EPSG:4326")


# create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_1')
visual_data.explore(column="amt", categorical=False, cmap='YlOrRd')

In [ ]:
visual_data = cty_m_sales[cty_m_sales['card_tpbuz_nm_2'].isin(top_cate_2)]

visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'CTY_NM'] = '부천시'
visual_data['geometry'] = visual_data['GEOMETRY'].apply(lambda wkt: loads(wkt))
visual_data = gpd.GeoDataFrame(visual_data, geometry = 'geometry').set_crs("EPSG:5181")
visual_data = visual_data.drop(columns = ['GEOMETRY'])
visual_data = visual_data.to_crs("EPSG:4326")

# create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_1')
visual_data.explore(column="amt", categorical=False, cmap='YlOrRd')

In [ ]:
visual_data = cty_s_sales[cty_s_sales['card_tpbuz_nm_3'].isin(top_cate_3)]

visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'CTY_NM'] = '부천시'
visual_data['geometry'] = visual_data['GEOMETRY'].apply(lambda wkt: loads(wkt))
visual_data = gpd.GeoDataFrame(visual_data, geometry = 'geometry').set_crs("EPSG:5181")
visual_data = visual_data.drop(columns = ['GEOMETRY'])
visual_data = visual_data.to_crs("EPSG:4326")

# create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_1')
visual_data.explore(column="amt", categorical=False, cmap='YlOrRd')

## 시군구별 성·연령별 업종별 매출액 평균

In [ ]:
def map_age_group(age):
    if age < 3:
        return '10대 미만'
    elif age < 5:
        return '10대'
    elif age < 7:
        return '20대'
    elif age < 9:
        return '30대'
    elif age < 11:
        return '40대'
    elif age < 13:
        return '50대'
    elif age < 15:
        return '60대'
    elif age < 17:
        return '70대'
    elif age < 19:
        return '80대'
    elif age < 21:
        return '90대'
    else:
      return '100세 이상'

# 'age' 구분값 변경
inflow_df['age'] = inflow_df['age'].apply(map_age_group)

# 시군구별 성,연령별 카드사 소분류 업종별 매출액 평균
cty_sales = inflow_df.groupby(['cty_rgn_no', 'sex', 'age', 'card_tpbuz_cd', 'card_tpbuz_nm_3'])['amt'].mean().reset_index()
cty_sas_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

# 시군구별 성,연령별 카드사 중분류 업종별 매출액 평균
cty_sales = inflow_df.groupby(['cty_rgn_no', 'sex', 'age', 'card_tpbuz_nm_2'])['amt'].mean().reset_index()
cty_sam_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

# 시군구별 성,연령별 카드사 대분류 업종별 매출액 평균
cty_sales = inflow_df.groupby(['cty_rgn_no', 'sex', 'age', 'card_tpbuz_nm_1'])['amt'].mean().reset_index()
cty_sal_sales = pd.merge(cty_sales, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

In [ ]:
# sido = top_sido[0]
# visual_data = cty_sal_sales[cty_sal_sales['SGG_NM'] == sido]
# cate = top_cate_1[1]
# visual_data = visual_data[visual_data['card_tpbuz_nm_1'] == cate]
visual_data

In [ ]:
visual_data_temp = cty_sal_sales[cty_sal_sales['SGG_NM'] == top_sido[0]]
visual_date = visual_data_temp[visual_data_temp['card_tpbuz_nm_1'] == top_cate_1[0]]
visual_date

In [ ]:
top = 3 # 상위 3개

# 가장 많은 상위 3개 카테고리
top_cate_1 = cty_sal_sales.groupby('card_tpbuz_nm_1')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_1'].to_list()[:top]
top_cate_2 = cty_sam_sales.groupby('card_tpbuz_nm_2')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_2'].to_list()[:top]
top_cate_3 = cty_sas_sales.groupby('card_tpbuz_nm_3')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_3'].to_list()[:top]
top_sido = cty_sal_sales.groupby('SGG_NM')['amt'].sum().sort_values(ascending=False).reset_index()['SGG_NM'].to_list()[:top]

In [ ]:
for sido in top_sido:
  visual_data_temp = cty_sal_sales[cty_sal_sales['SGG_NM'] == sido]
  for cate in top_cate_1:
    visual_date = visual_data_temp[visual_data_temp['card_tpbuz_nm_1'] == cate]
    # create_lineplot(visual_data, 'hour2', 'amt', title=f'{sido} 업종별(TOP {top}) 평균 매출액', xlabel='시간', ylabel='평균 매출액', legend='card_tpbuz_nm_1')
    create_barplot(visual_date, 'age', 'amt',  title=f'{sido} {cate}업종 연령별 카드사 평균 매출액', xlabel='연령별', ylabel='평균 매출액', legend='sex')

## 요일별카드사 업종별 매출액 평균

In [ ]:
week_dic = {
  1 : '월',
  2 : '화',
  3 : '수',
  4 : '목',
  5 : '금',
  6 : '토',
  7 : '일'
}

inflow_df['day_2'] = inflow_df['day'].map(week_dic)

# 시군구별 요일별 카드사 소분류 업종별 매출액 평균
cty_day = inflow_df.groupby(['cty_rgn_no', 'day_2', 'card_tpbuz_cd', 'card_tpbuz_nm_3'])['amt'].mean().reset_index()
cty_day_s = pd.merge(cty_day, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

# 시군구별 요일별 카드사 중분류 업종별 매출액 평균
cty_day = inflow_df.groupby(['cty_rgn_no', 'day_2', 'card_tpbuz_nm_2'])['amt'].mean().reset_index()
cty_day_m = pd.merge(cty_day, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

# 시군구별 요일별 카드사 대분류 업종별 매출액 평균
cty_day = inflow_df.groupby(['cty_rgn_no', 'day_2', 'card_tpbuz_nm_1'])['amt'].mean().reset_index()
cty_day_l = pd.merge(cty_day, cty_shp, left_on='cty_rgn_no', right_on='SGG_CD', how='left')

In [ ]:
top = 3 # 상위 3개

# 가장 많은 상위 3개 카테고리
top_cate_1 = cty_day_l.groupby('card_tpbuz_nm_1')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_1'].to_list()[:top]
top_cate_2 = cty_day_m.groupby('card_tpbuz_nm_2')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_2'].to_list()[:top]
top_cate_3 = cty_day_s.groupby('card_tpbuz_nm_3')['amt'].sum().sort_values(ascending=False).reset_index()['card_tpbuz_nm_3'].to_list()[:top]
top_sido = cty_day_l.groupby('SGG_NM')['amt'].sum().sort_values(ascending=False).reset_index()['SGG_NM'].to_list()[:top]

In [ ]:
for sido in top_sido:
  visual_data_temp = cty_day_l[cty_day_l['SGG_NM'] == sido]
  visual_data = visual_data_temp[visual_data_temp['card_tpbuz_nm_1'].isin(top_cate_1)]
  create_lineplot(visual_data, 'day_2', 'amt', title=f'{sido} 대분류 업종별(TOP {top}) 평균 매출액', xlabel='요일', ylabel='평균 매출액', legend='card_tpbuz_nm_1')

In [ ]:
for sido in top_sido:
  visual_data_temp = cty_day_m[cty_day_m['SGG_NM'] == sido]
  visual_data = visual_data_temp[visual_data_temp['card_tpbuz_nm_2'].isin(top_cate_2)]
  create_lineplot(visual_data, 'day_2', 'amt', title=f'{sido} 중분류 업종별(TOP {top}) 평균 매출액', xlabel='요일', ylabel='평균 매출액', legend='card_tpbuz_nm_2')

In [ ]:
for sido in top_sido:
  visual_data_temp = cty_day_s[cty_day_s['SGG_NM'] == sido]
  visual_data = visual_data_temp[visual_data_temp['card_tpbuz_nm_3'].isin(top_cate_3)]
  create_lineplot(visual_data, 'day_2', 'amt', title=f'{sido} 소분류 업종별(TOP {top}) 평균 매출액', xlabel='요일', ylabel='평균 매출액', legend='card_tpbuz_nm_3')

#기업신용

In [ ]:
col = ["stdr_ym", "unq_cd", "entrps_nm", "bizno", "crpno", "eprdatastsdivcd", "eprmdydivcd", "scaledivcd", "eprdtldivcd", "ltgmktdivcd", "amnisuyn",
       "etl_ipc_yn", "comp_fnd_date", "comp_close", "emp_pay_YM", "employees", "pay_avg", "mm_clby_gjob_tnpnum", "mmly_rtm_tnpnum", "loc_zip",
       "sido_nm", "sigun_nm", "admi_nm", "admi2_nm", "gis_x", "gis_y", "noaddr", "rdnmaddr", "induty_pri_cd", "induty_pri_nm", "induty_med_cd", "induty_med_nm",
       "watch_grade", "acct_dt", "pi_captial", "kiscon", "repr", "repr_birth", "cert_venture", "cert_innobiz", "cert_mainbiz", "cert_rndcntr", "cert_rnddpt",
       "cert_net", "cert_nep", "cert_mct"]

hqmaster = pd.read_csv('/content/drive/MyDrive/data/nice데이터/기업데이터/202401/gg_corp1_hqmaster_202401.txt',
                   encoding='cp949',
                   sep='|',
                   header=None,
                   names=col)
cty_shp = pd.read_csv('/content/drive/MyDrive/시군구 경계/TBSHP_LH_CTY_202404081545.csv') # 시군구 경계 load
cty_shp['geometry'] = cty_shp['GEOMETRY'].apply(lambda wkt: loads(wkt))
cty_shp = gpd.GeoDataFrame(cty_shp, geometry = 'geometry').set_crs("EPSG:5181")
# cty_shp = cty_shp.drop(columns = ['GEOMETRY'])
cty_shp = cty_shp.to_crs("EPSG:4326")

In [ ]:
#geodataframe 변환
hqmaster_gdf = gpd.GeoDataFrame(hqmaster, geometry=gpd.points_from_xy(hqmaster.gis_x, hqmaster.gis_y)).set_crs("EPSG:5181")
hqmaster_gdf = hqmaster_gdf.to_crs("EPSG:4326")
hqmaster_gdf = gpd.sjoin(hqmaster_gdf, cty_shp, how='left', predicate='within').to_crs("EPSG:4326")
hqmaster_gdf = hqmaster_gdf.dropna(subset=['GEOMETRY'])
hqmaster_gdf = hqmaster_gdf.drop(columns=['geometry'])

hqmaster_gdf['gg_check'] = hqmaster_gdf['SGG_CD'].astype(str).str.slice(0,2)
hqmaster_gdf = hqmaster_gdf[hqmaster_gdf['gg_check'] == '41']
hqmaster_gdf['scaledivcd'] = hqmaster_gdf['scaledivcd'].replace({1: '대기업', 2: '중소기업', 3: '중견기업', 0: '대상아님'})

## 시군구별 기업 규모별 업체수

In [ ]:
hqmaster_gdf['scaledivcd'] = hqmaster_gdf['scaledivcd'].replace({1: '대기업', 2: '중소기업', 3: '중견기업', 0: '대상아님'})
visual_data = hqmaster_gdf[hqmaster_gdf['scaledivcd'] == '대기업']
visual_data = visual_data.groupby(['SGG_CD', 'SGG_NM', 'GEOMETRY', 'scaledivcd'])['entrps_nm'].count().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'SGG_CD'] = 41190
visual_data['geometry'] = visual_data['GEOMETRY'].apply(lambda wkt: loads(wkt))
visual_data = gpd.GeoDataFrame(visual_data, geometry = 'geometry').set_crs("EPSG:5181")
visual_data = visual_data.drop(columns = ['GEOMETRY'])
visual_data = visual_data.to_crs("EPSG:4326")
visual_data.explore(column="entrps_nm", categorical=False, cmap='YlOrRd')

In [ ]:
visual_data = hqmaster_gdf[hqmaster_gdf['scaledivcd'] == '중소기업']
visual_data = visual_data.groupby(['SGG_CD', 'SGG_NM', 'GEOMETRY', 'scaledivcd'])['entrps_nm'].count().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'SGG_CD'] = 41190
visual_data['geometry'] = visual_data['GEOMETRY'].apply(lambda wkt: loads(wkt))
visual_data = gpd.GeoDataFrame(visual_data, geometry = 'geometry').set_crs("EPSG:5181")
visual_data = visual_data.drop(columns = ['GEOMETRY'])
visual_data = visual_data.to_crs("EPSG:4326")
visual_data.explore(column="entrps_nm", categorical=False, cmap='YlOrRd')

In [ ]:
hqmaster_gdf['scaledivcd'] = hqmaster_gdf['scaledivcd'].replace({1: '대기업', 2: '중소기업', 3: '중견기업', 0: '대상아님'})
visual_data = hqmaster_gdf[hqmaster_gdf['scaledivcd'] == '중견기업']
visual_data = visual_data.groupby(['SGG_CD', 'SGG_NM', 'GEOMETRY', 'scaledivcd'])['entrps_nm'].count().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'SGG_CD'] = 41190
visual_data['geometry'] = visual_data['GEOMETRY'].apply(lambda wkt: loads(wkt))
visual_data = gpd.GeoDataFrame(visual_data, geometry = 'geometry').set_crs("EPSG:5181")
visual_data = visual_data.drop(columns = ['GEOMETRY'])
visual_data = visual_data.to_crs("EPSG:4326")
visual_data.explore(column="entrps_nm", categorical=False, cmap='YlOrRd')

## 시군구별(TOP 5) 기업 규모별 입사자/퇴사자 수


In [ ]:
# 시군구별 기업규모별 평균 종업원수, 입사자수, 퇴사자수
# h_emp = hqmaster_gdf.groupby(['sigun_nm', 'scaledivcd'])[['employees', 'mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()

visual_data = hqmaster_gdf[hqmaster_gdf['scaledivcd'] == '대기업']
visual_data = visual_data.groupby(['SGG_CD', 'SGG_NM', 'GEOMETRY', 'scaledivcd'])[['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'SGG_CD'] = 41190

visual_data = visual_data.melt(id_vars=['SGG_CD', 'SGG_NM'],
                    value_vars=['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum'])
top_5_visual_data = visual_data.sort_values(['value'], ascending=False)
top_5__list = top_5_visual_data['SGG_NM'].unique().tolist()[:5]
top_5_visual_data = top_5_visual_data[top_5_visual_data['SGG_NM'].isin(top_5__list)]
top_5_visual_data.loc[top_5_visual_data['variable'] == 'mm_clby_gjob_tnpnum', 'variable'] = '입사자'
top_5_visual_data.loc[top_5_visual_data['variable'] == 'mmly_rtm_tnpnum', 'variable'] = '퇴사자'

create_barplot(top_5_visual_data, 'SGG_NM', 'value',  title=f'대기업 시군구별 평균 입사자/퇴사자 수', xlabel='시군구', ylabel='수', legend='variable')

In [ ]:
# 시군구별 기업규모별 평균 종업원수, 입사자수, 퇴사자수
# h_emp = hqmaster_gdf.groupby(['sigun_nm', 'scaledivcd'])[['employees', 'mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()

visual_data = hqmaster_gdf[hqmaster_gdf['scaledivcd'] == '중소기업']
visual_data = visual_data.groupby(['SGG_CD', 'SGG_NM', 'GEOMETRY', 'scaledivcd'])[['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'SGG_NM'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'SGG_CD'] = 41190

visual_data = visual_data.melt(id_vars=['SGG_CD', 'SGG_NM'],
                    value_vars=['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum'])
top_5_visual_data = visual_data.sort_values(['value'], ascending=False)
top_5__list = top_5_visual_data['SGG_NM'].unique().tolist()[:5]
top_5_visual_data = top_5_visual_data[top_5_visual_data['SGG_NM'].isin(top_5__list)]
top_5_visual_data.loc[top_5_visual_data['variable'] == 'mm_clby_gjob_tnpnum', 'variable'] = '입사자'
top_5_visual_data.loc[top_5_visual_data['variable'] == 'mmly_rtm_tnpnum', 'variable'] = '퇴사자'

create_barplot(top_5_visual_data, 'SGG_NM', 'value',  title=f'중소기업 시군구별 평균 입사자/퇴사자 수', xlabel='시군구', ylabel='수', legend='variable')

In [ ]:
   # 시군구별 기업규모별 평균 종업원수, 입사자수, 퇴사자수
# h_emp = hqmaster_gdf.groupby(['sigun_nm', 'scaledivcd'])[['employees', 'mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()

visual_data = hqmaster_gdf[hqmaster_gdf['scaledivcd'] == '중견기업']
visual_data = visual_data.groupby(['SGG_CD', 'SGG_NM', 'GEOMETRY', 'scaledivcd'])[['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'sigun_nm'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'sigun_nm'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'cty_rgn_no'] = 41190

visual_data = visual_data.melt(id_vars=['SGG_CD', 'SGG_NM'],
                    value_vars=['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum'])
top_5_visual_data = visual_data.sort_values(['value'], ascending=False)
top_5__list = top_5_visual_data['SGG_NM'].unique().tolist()[:5]
top_5_visual_data = top_5_visual_data[top_5_visual_data['SGG_NM'].isin(top_5__list)]
top_5_visual_data.loc[top_5_visual_data['variable'] == 'mm_clby_gjob_tnpnum', 'variable'] = '입사자'
top_5_visual_data.loc[top_5_visual_data['variable'] == 'mmly_rtm_tnpnum', 'variable'] = '퇴사자'

create_barplot(top_5_visual_data, 'SGG_NM', 'value',  title=f'중견기업 시군구별 평균 입사자/퇴사자 수', xlabel='시군구', ylabel='수', legend='variable')

## 시군구별(TOP 5) 대분류(TOP 5) 업종별 평균 급료

In [ ]:
visual_data = hqmaster_gdf.groupby(['SGG_CD', 'SGG_NM', 'induty_pri_nm'])[['pay_avg']].mean().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'sigun_nm'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'sigun_nm'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'cty_rgn_no'] = 41190

top_5_visual_data = visual_data.sort_values(['pay_avg'], ascending=False)
cate_top_5_list = top_5_visual_data['induty_pri_nm'].drop_duplicates().tolist()[:5]
sido_top_5_list = top_5_visual_data['SGG_NM'].drop_duplicates().tolist()[:5]

cate_sido_top5_data = top_5_visual_data[top_5_visual_data['induty_pri_nm'].isin(cate_top_5_list)]
cate_sido_top5_data = cate_sido_top5_data[cate_sido_top5_data['SGG_NM'].isin(sido_top_5_list)]
cate_sido_top5_data['pay_avg'] = cate_sido_top5_data['pay_avg'].fillna(0)
create_barplot(cate_sido_top5_data, 'SGG_NM', 'pay_avg',  title=f'시군구별(TOP 5) 대분류(TOP 5) 업종별 평균 급료', xlabel='시군구', ylabel='평균 급료', legend='induty_pri_nm')

## 시군구(TOP 5) 대분류 업종별(TOP 5) 평균 입사자/퇴사자 수

In [ ]:
# 시군구별 대분류 업종별 평균 종업원수, 입사자수, 퇴사자수
h_pemp = hqmaster_gdf.groupby(['SGG_CD', 'SGG_NM', 'induty_pri_nm'])[['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()
cty_h_pemp = pd.merge(h_pemp, cty_shp, left_on='sigun_nm', right_on='SGG_NM', how='left')
cty_h_pemp = cty_h_pemp[['SGG_CD', 'sigun_nm', 'induty_pri_cd', 'induty_pri_nm', 'employees', 'mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum', 'GEOMETRY']]

In [ ]:
visual_data = hqmaster_gdf.groupby(['SGG_CD', 'SGG_NM', 'induty_pri_nm'])[['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum']].mean().reset_index()
visual_data.loc[visual_data['SGG_NM'].isin(['부천시 소사구', '부천시 원미구', '부천시 오정구']), 'sigun_nm'] = '부천시'
visual_data.loc[visual_data['SGG_NM'].isin(['부천시소사구', '부천시원미구', '부천시오정구']), 'sigun_nm'] = '부천시'
visual_data.loc[visual_data['SGG_CD'].isin([41194, 41192, 41196]), 'cty_rgn_no'] = 41190
visual_data['mm_clby_gjob_tnpnum'] = visual_data['mm_clby_gjob_tnpnum'].fillna(0)
visual_data['mmly_rtm_tnpnum'] = visual_data['mmly_rtm_tnpnum'].fillna(0)
visual_data['total'] = visual_data['mm_clby_gjob_tnpnum'] + visual_data['mmly_rtm_tnpnum']
visual_data = visual_data.sort_values(['total'], ascending=False)

top_5_sido_list = visual_data['SGG_NM'].drop_duplicates().tolist()[:5]
top_5_cate_list = visual_data['induty_pri_nm'].drop_duplicates().tolist()[:5]

visual_top5_data = visual_data[visual_data['SGG_NM'].isin(top_5_sido_list)]
visual_top5_data = visual_top5_data[visual_top5_data['induty_pri_nm'].isin(top_5_cate_list)]

visual_top5_data = visual_top5_data.melt(id_vars=['SGG_CD', 'SGG_NM', 'induty_pri_nm'],
                    value_vars=['mm_clby_gjob_tnpnum', 'mmly_rtm_tnpnum'])
visual_top5_data = visual_top5_data.sort_values(['value'], ascending=False)

visual_top5_data.loc[visual_top5_data['variable'] == 'mm_clby_gjob_tnpnum', 'variable'] = '입사자'
visual_top5_data.loc[visual_top5_data['variable'] == 'mmly_rtm_tnpnum', 'variable'] = '퇴사자'

# visual_top5_data

for cate in top_5_cate_list:
  visual_data_temp = visual_top5_data[visual_top5_data['induty_pri_nm'] == cate]
  create_barplot(visual_data_temp, 'SGG_NM', 'value',  title=f'{cate} 업종 시군구별 입사자/퇴사자 평균수', xlabel='시군구', ylabel='평균 수', legend='variable')



## 시군구별 업종별 신설법인 기업수

In [ ]:
col = ['stdr_ym', 'sido_nm', 'sigun_nm', 'admi_nm', 'induty_pri_cd', 'induty_pri_nm', 'induty_med_cd', 'induty_med_nm', 'ncr_crp_comp_cn']

new_comp = pd.read_csv('/content/drive/MyDrive/data/nice데이터/기업데이터/202401/gg_corp4_new_202401.txt',
                   encoding='cp949',
                   sep='|',
                   header=None,
                   names=col)
new_comp

In [ ]:
# 시군구별 대분류 업종별 신설법인 기업수
cty_new_pcomp = new_comp.groupby(['sigun_nm', 'induty_pri_cd', 'induty_pri_nm'])['ncr_crp_comp_cn'].sum().reset_index()
cty_new_pcomp = pd.merge(cty_new_pcomp, cty_shp, left_on='sigun_nm', right_on='SGG_NM', how='left')
cty_new_pcomp = cty_new_pcomp[['SGG_CD', 'sigun_nm',  'induty_pri_cd', 'induty_pri_nm', 'ncr_crp_comp_cn', 'GEOMETRY']]

# 시군구별 중분류 업종별 신설법인 기업수
cty_new_mcomp = new_comp.groupby(['sigun_nm',  'induty_med_cd', 'induty_med_nm'])['ncr_crp_comp_cn'].sum().reset_index()
cty_new_mcomp = pd.merge(cty_new_mcomp, cty_shp, left_on='sigun_nm', right_on='SGG_NM', how='left')
cty_new_mcomp = cty_new_mcomp[['SGG_CD', 'sigun_nm',  'induty_med_cd', 'induty_med_nm', 'ncr_crp_comp_cn', 'GEOMETRY']]

# 개인신용

In [ ]:
chunk_size = 1000000
chunks = []

# use_col = ['cty_rgn_no', 'admi_cty_no', 'card_tpbuz_cd', 'card_tpbuz_nm_1', 'card_tpbuz_nm_2', 'card_tpbuz_nm_3', 'inflow_cd', 'sex', 'age', 'day', 'amt', 'cnt']

# 법정동리단위 소득, 소비, 부채, 인구통계 데이터 load
for chunk in pd.read_csv('/content/drive/MyDrive/data/nice데이터/개인데이터/NICE평가정보_개인신용익명정보 산출파일_202401_202402.zip (Unzipped Files)/GYEONGGI_BCD_STAT_2401.csv',
                         encoding = 'euc-kr',
                         chunksize=chunk_size):
                        #  usecols = use_col,
                        #  chunksize=chunk_size):
    chunks.append(chunk)

stat_df = pd.concat(chunks, ignore_index=True) # 유입지별 매출 데이터 load
stat_df

## 시군구별 데이터 분석

In [ ]:
# age 10대 단위로 매핑
def map_age_group(age):
    if age < 20:
        return '10대'
    elif age < 30:
        return '20대'
    elif age < 40:
        return '30대'
    elif age < 50:
        return '40대'
    elif age < 60:
        return '50대'
    elif age < 70:
        return '60대'
    else:
      return '70대 이상'

# 'age' 구분값 변경
stat_df['AGE'] = stat_df['AGE'].apply(map_age_group)

# 행정동 컬럼 생성
stat_df['ADMI'] = stat_df['BCD'].astype(str).str[:8].astype(int)

# 시군구 컬럼 생성
stat_df['CTY'] = stat_df['BCD'].astype(str).str[:5].astype(int)

# 행정동별 연령별 월소득평균
bcd_age = stat_df.groupby(['ADMI', 'AGE'])['AVG_INC'].mean().reset_index()
admi_bcd_age = pd.merge(bcd_age, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 행정동별 연령별 신용카드 이용금액 평균
bcd_cred = stat_df.groupby(['ADMI', 'AGE'])['AVG_CRED'].mean().reset_index()
admi_bcd_cred = pd.merge(bcd_cred, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 행정동별 연령별 체크카드 이용금액 평균
bcd_chk = stat_df.groupby(['ADMI', 'AGE'])['AVG_CHECK'].mean().reset_index()
admi_bcd_chk = pd.merge(bcd_chk, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 행정동별 연령별 카드 총 이용금액 평균
bcd_card = stat_df.groupby(['ADMI', 'AGE'])['AVG_CARD'].mean().reset_index()
admi_bcd_card = pd.merge(bcd_card, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 시군구별 연령별 대출, 은행업권대출, 비은행업권대출, 신용대출, 주택담보대출, 예적금 유가 담보대출, 정책자금 대출 보유자 수
bcd_loan = stat_df.groupby(['ADMI', 'AGE'])['LOAN_CNT', 'LOAN1_CNT', 'LOAN2_CNT', 'CRED_LOAN_CNT', 'HOUS_LOAN_CNT', 'BANK_LOAN_CNT', 'POL_LOAN_CNT'].sum().reset_index()
admi_bcd_loan = pd.merge(bcd_loan, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 행정동별 연령별 신용평점 평균
bcd_score = stat_df.groupby(['ADMI', 'AGE'])['AVG_SCORE'].mean().reset_index()
admi_bcd_score = pd.merge(bcd_score, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

In [ ]:
# 시군구별 연령별 월소득평균
bcd_age = stat_df.groupby(['CTY', 'AGE'])['AVG_INC'].mean().reset_index()
cty_bcd_age = pd.merge(bcd_age, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 신용카드 이용금액 평균
bcd_cred = stat_df.groupby(['CTY', 'AGE'])['AVG_CRED'].mean().reset_index()
cty_bcd_cred = pd.merge(bcd_cred, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 체크카드 이용금액 평균
bcd_chk = stat_df.groupby(['CTY', 'AGE'])['AVG_CHECK'].mean().reset_index()
cty_bcd_chk = pd.merge(bcd_chk, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 카드 총 이용금액 평균
bcd_card = stat_df.groupby(['CTY', 'AGE'])['AVG_CARD'].mean().reset_index()
cty_bcd_card = pd.merge(bcd_card, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 대출, 은행업권대출, 비은행업권대출, 신용대출, 주택담보대출, 예적금 유가 담보대출, 정책자금 대출 보유자 수
bcd_loan = stat_df.groupby(['CTY', 'AGE'])['LOAN_CNT', 'LOAN1_CNT', 'LOAN2_CNT', 'CRED_LOAN_CNT', 'HOUS_LOAN_CNT', 'BANK_LOAN_CNT', 'POL_LOAN_CNT'].sum().reset_index()
cty_bcd_loan = pd.merge(bcd_loan, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 신용평점 평균
bcd_score = stat_df.groupby(['CTY', 'AGE'])['AVG_SCORE'].mean().reset_index()
cty_bcd_score = pd.merge(bcd_score, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

In [ ]:
# 시군구별 연령별 아파트, 비아파트 거주자수
apt_nonapt = stat_df.groupby(['CTY', 'AGE'])['APT_CNT', 'NAPT_CNT'].sum().reset_index()
cty_apt = pd.merge(apt_nonapt, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 자가, 비자가 거주자수
own_rent = stat_df.groupby(['CTY', 'AGE'])['OWN_HOUS_CNT', 'NOWN_HOUS_CNT'].sum().reset_index()
cty_own = pd.merge(own_rent, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

# 시군구별 연령별 평균 출퇴근 거리
dist = stat_df.groupby(['CTY', 'AGE'])['AVG_DIST'].mean().reset_index()
cty_dist = pd.merge(dist, cty_shp, left_on='CTY', right_on='SGG_CD', how='left')

## 전입데이터(서울->경기도) 분석

In [ ]:
# 전입
in_df = pd.read_csv('/content/drive/MyDrive/data/nice데이터/개인데이터/NICE평가정보_개인신용익명정보 산출파일_202401_202402.zip (Unzipped Files)/GYEONGGI_IN_STAT_2401.csv', encoding='euc-kr')
# 기준시점 시도 컬럼 생성
in_df['BS_SIDO'] = in_df['BS_SIGNGU'].astype(str).str[:2]
# 행정동 컬럼 생성
in_df['ADMI'] = in_df['BS_HCD'].astype(str).str[:8].astype(int)

In [ ]:
# 서울특별시에서 경기도로 전입한 사람들의 월소득 평균 (행정동 단위)
sg = in_df[(in_df['BF_SIDO'] == 11) & (in_df['BS_SIDO'] == '41')]
sg_inc = sg.groupby(['BF_SIDO', 'BS_SIDO', 'ADMI'])['AVG_INC'].mean().reset_index()
admi_inc = pd.merge(sg_inc, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 서울특별시에서 경기도로 전입한 사람들의 카드 총 이용금액 평균 (행정동 단위)
sg_card = sg.groupby(['BF_SIDO', 'BS_SIDO', 'ADMI'])['AVG_CARD'].mean().reset_index()
admi_card = pd.merge(sg_card, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 서울특별시에서 경기도로 전입한 사람들의 대출 평균 잔액 (행정동 단위)
sg_loan = sg.groupby(['BF_SIDO', 'BS_SIDO', 'ADMI'])['AVG_LOAN'].mean().reset_index()
admi_loan = pd.merge(sg_loan, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

## 전출데이터(경기도->서울) 분석

In [ ]:
# 전출
out_df = pd.read_csv('/content/drive/MyDrive/data/nice데이터/개인데이터/NICE평가정보_개인신용익명정보 산출파일_202401_202402.zip (Unzipped Files)/GYEONGGI_OUT_STAT_2401.csv', encoding='euc-kr')
# 기준시점 시도 컬럼 생성
out_df['BS_SIDO'] = out_df['BS_SIGNGU'].astype(str).str[:2]
# 행정동 컬럼 생성
out_df['ADMI'] = out_df['BS_HCD'].astype(str).str[:8].astype(int)

In [ ]:
# 경기도에서 서울특별시로 월소득 평균 (행정동 단위)
gs = out_df[(out_df['BS_SIDO'] == '41') & (out_df['AF_SIDO'] == 11)]
gs_inc = gs.groupby(['BS_SIDO', 'AF_SIDO','ADMI'])['AVG_INC'].mean().reset_index()
admi_inc = pd.merge(gs_inc, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 경기도에서 서울특별시로 카드 총 이용금액 평균 (행정동 단위)
gs_card = gs.groupby(['BS_SIDO', 'AF_SIDO','ADMI'])['AVG_CARD'].mean().reset_index()
admi_card = pd.merge(gs_card, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')

# 경기도에서 서울특별시로 대출 평균 잔액 (행정동 단위)
gs_loan = gs.groupby(['BS_SIDO', 'AF_SIDO','ADMI'])['AVG_LOAN'].mean().reset_index()
admi_loan = pd.merge(gs_loan, admi_shp, left_on='ADMI', right_on='ADMI_CD', how='left')